In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [2]:
#import torch
#print("CUDA version:", torch.version.cuda)

In [3]:
!pip install torch==2.4.1+cu124 torchaudio==2.4.1+cu124 torchvision==0.19.1+cu124 --index-url https://download.pytorch.org/whl/cu124 xformers

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207

In [4]:
from IPython.display import clear_output

In [ ]:
!pip install pytorch_lightning
clear_output()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.7 MB/s eta 0:00:00


In [ ]:
!pip install kornia

In [ ]:
!pip install wget

In [ ]:
pip install ptflops

In [ ]:
import os
import wget

In [ ]:
import torch.nn as nn
import torch

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torch.nn.functional import scaled_dot_product_attention
import torch.nn.functional as F

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger

import pytorch_lightning as pl

import torchmetrics
from torchmetrics import Metric, MetricCollection

from transformers import ViTConfig, ViTForImageClassification

In [ ]:
pip install fvcore


In [ ]:
from einops import rearrange

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
geometric_dimension = 16

global_var = {
    # Data
    'data_path': "/content/vessel_data",
    'prefix': "https://drive.google.com/uc?export=",
    'bifurcating_data_id': "download&id=1yFBcEpJzIwllwqOLzJJzIKmC54GjD6Fs",
    'single_data_id': "download&id=19AHhDU1UWBzpG33wlB1r7b6sH3lMVly5",
    'guidance_id': "download&id=1cDI0Eh6hBpHQdE55-Ke0XSpUxcVu0SO9",
    'outer_id': "download&id=1nu_eI2SyN4zWRhIDJTsKxBW4b8eq6fPk",
    'current_name': "bifurcating",

    # Datasets
    'single_path': "./single/single.hdf5",
    'bifurcating_path': "./bifurcating/bifurcating.hdf5",
    'dataset_path': "./dataset.hdf5",
    'train_path': "./train.hdf5",
    'test_path': "./test.hdf5",
    'data_crop': 50,

    # Datasets quality check
    'data_length': 2000,
    'bases': 3,
    'data_keys': ['face', 'inlet_idcs', 'pos', 'pressure', 'wss'],

    # Geometric algebra
    'ga_dimension': geometric_dimension,
    'grade_components': [1,3,3,1] if geometric_dimension == 8 else [1,4,6,4,1],
    'ga_permutation': [[[0,1]],[[2,3]],[[4,5]], [[6,7]]] if geometric_dimension == 8 else
        [
          [[0,1]],
          [[2,5],[3,6],[4,7]],
          [[8,11],[9,12],[10,13]],
          [[14,15]]
        ],

    # Hyperparameter search
    'optuna_trial': 3,
    'sampler_seed': 42,

    # Train
    'baseline_in_channels': 10,
    'train_test_split': 0.8,
    'validation_split': 0.1,
    'batch_size': 32,
    'num_workers': 1,
    'hidden_dim': 16,
    'n_heads': 4,
    'classes': 10,
    'input_channels': 4,
    'kernel_size': 3,
    'epochs': 10
}

In [ ]:
from datetime import datetime

# Prendi la data corrente
now = datetime.now()

# Formatta giorno e mese, ad esempio "14-06" per 14 giugno
date_str = now.strftime("%d-%m")
loop_counter = 0

# Geometric Algebra
This code comprises a set of functions for processing and manipulating multivectors in the context of algebraic geometry. It builds a framework for handling geometric data in a multidimensional space using algebraic geometric principles

The functions include embedding operations for translating position, face, wall shear stress, inlet, and pressure vectors into multivectors. Additionally, there is a blade operator construction, which appears to be pivotal for subsequent geometric operations.

The code implements reversal, dualization, and various geometric product operations, such as inner, outer, and sandwich products. These operations are designed to work efficiently with multivectors and utilize PyTorch for tensor computations.

To deepen and better apply the concepts of geometric algebra [1,2,4] have proven to be valuable resources.

## Embedding in geometric algebra

This section deals with bringing the raw data of the dataset into a projective 3D algebraic geometry, following the table defined in [1]. The geometric algebra that is the subject of this project consists of four bases $e_0,e_1,e_2,e_3$, which together represent the coordinates of multivectors in 3D projective geometry $\mathcal{G}_{3,0,1}$. The subscript numbers indicate that there are three degrees of freedom for displacement ($d = 3$) in space and one more projective degree of freedom, respectively. Depending on how many coordinates are used, the degree $k$ of the multivector changes:

* **Grade 0** (1): Scalar
* **Grade 1** ($e_0, e_i$): Vector
* **Grade 2** ($e_{0i}, e_{ij}$): Bivector
* **Grade 3** ($e_{0ij}, e_{123}$): Trivector
* **Grade 4** ($e_{0123}$): Pseudoscalar

Based on these coordinates, combinations can be made that realize multivectors composed of various table elements. Therefore, the dimension of the geometric algebra is equal to $\sum_{i=0}^{d+1}\binom{d}{i} = 2^{d+1} = 16$

![mv_table](https://drive.google.com/uc?export=download&id=1xgjdepW0Z91cx_8cG_5M_EC2rdJsTpOL)

In [ ]:
import torch
import torch.nn.functional as F

def get_planes_min_max_mv(image, patch_size, mv_dimension, device="cpu"):
    # Sposta l'immagine sul device specificato
    image = image.to(device)

    batch_size, channels, height, width = image.shape
    image_norm = torch.norm(image, dim=1)

    points = torch.zeros(batch_size, height, width, 3, 3, device=device)

    # Creazione del kernel medio e spostamento su device
    mean_kernel = torch.full((patch_size, patch_size), 1/(patch_size**2), dtype=torch.float32, device=device)
    mean_kernel = mean_kernel.expand(3, 1, 3, 3).to(device)

    # Operazioni convolutive su device
    mean = F.conv2d(image, mean_kernel, padding=1, groups=3)
    max = F.max_pool2d(image, kernel_size=patch_size, stride=1, padding=patch_size // 2)
    min = F.max_pool2d(-image, kernel_size=patch_size, stride=1, padding=patch_size // 2)

    points[..., 0, :] = mean.permute(0, 2, 3, 1)
    points[..., 1, :] = max.permute(0, 2, 3, 1)
    points[..., 2, :] = min.permute(0, 2, 3, 1)

    p1 = points[..., 0, :]
    p2 = points[..., 1, :]
    p3 = points[..., 2, :]

    v1 = p2 - p1
    v2 = p3 - p1

    normal = torch.cross(v1, v2, dim=-1)
    normal = normal / (torch.norm(normal, dim=-1, keepdim=True) + 1e-8)

    # Creazione del tensore planes su device
    planes = torch.zeros(batch_size, height, width, mv_dimension, device=device)
    planes[..., 1] = -torch.sum(normal * points[..., 0, :], dim=-1)
    planes[..., 2] = normal[..., 0]
    planes[..., 3] = normal[..., 1]
    planes[..., 4] = normal[..., 2]

    return planes


In [ ]:
def get_planes_PCA(image, patch_size, mv_dimension):
    # Sposta l'immagine sul device specificato

    B, C, H, W = image.shape

    padding = patch_size // 2  # Padding per mantenere la dimensione originale

    # Unfold estrae finestre (B, C*P, H*W)
    patches = F.unfold(image, kernel_size=patch_size, padding=padding)

    # Reshape per ottenere (B, C, P, H, W)
    P = patch_size * patch_size  # Numero di pixel nella finestra
    patches = patches.view(B, C, P, H, W)  # (B, C, P, H, W)
    patches = patches.permute(0, 3, 4, 2, 1)

    B, H, W, P, C = patches.shape

    # Centriamo i dati sottraendo la media su ogni finestra
    mean = patches.mean(dim=2, keepdim=True)  # (B, H, W, 1, C)
    centered_patches = patches - mean  # Sottrae la media dai colori locali

    # SVD su ogni finestra (B, H, W, P, C) → U, S, Vh
    U, S, Vh = torch.linalg.svd(centered_patches, full_matrices=False)  # Vh ha shape (B, H, W, C, C)

    # Prendiamo solo i primi 3 autovettori (primi 3 righe di Vh)
    normal = Vh[:, :, :, -1, :]  # (B, H, W, 3, C)

    # Creazione del tensore planes su device
    planes = torch.zeros(B, H, W, mv_dimension, device=device)
    planes[..., 1] = -torch.sum(normal * image.permute(0, 2, 3, 1), dim=-1)
    planes[..., 2] = normal[..., 0]
    planes[..., 3] = normal[..., 1]
    planes[..., 4] = normal[..., 2]

    return planes

In [ ]:
import torch
import torch.nn.functional as F

def compute_gradients(image, device="cpu"):
    # Sposta l'immagine sul device specificato
    image = image.to(device)

    # Definizione dei filtri Sobel per il gradiente in x e y su device
    sobel_x = torch.tensor([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]], dtype=torch.float32, device=device)

    sobel_y = torch.tensor([[-1, -2, -1],
                            [ 0,  0,  0],
                            [ 1,  2,  1]], dtype=torch.float32, device=device)

    # Espandiamo i filtri per coprire tutti i canali (R, G, B)
    sobel_x = sobel_x.expand(3, 1, 3, 3).to(device)  # (3, 1, 3, 3)
    sobel_y = sobel_y.expand(3, 1, 3, 3).to(device)  # (3, 1, 3, 3)

    # Calcolo del gradiente in x e y per ogni canale
    grad_x = F.conv2d(image, sobel_x, padding=1, groups=3)  # (B, 3, H, W)
    grad_y = F.conv2d(image, sobel_y, padding=1, groups=3)  # (B, 3, H, W)

    return grad_x, grad_y, sobel_x, sobel_y


In [ ]:
def compute_diagonal_gradients(image, device="cpu"):
    image = image.to(device)

    # Filtri Sobel diagonali
    sobel_diag1 = torch.tensor([[ 2,  1,  0],  # ↘ diagonale principale
                                [ 1,  0, -1],
                                [ 0, -1, -2]], dtype=torch.float32, device=device)

    sobel_diag2 = torch.tensor([[ 0,  1,  2],  # ↙ diagonale opposta
                                [ -1,  0,  1],
                                [ -2, -1,  0]], dtype=torch.float32, device=device)

    # Espandiamo i filtri per coprire tutti i canali (RGB)
    sobel_diag1 = sobel_diag1.expand(3, 1, 3, 3).to(device)
    sobel_diag2 = sobel_diag2.expand(3, 1, 3, 3).to(device)

    # Calcolo dei gradienti nelle due direzioni diagonali
    grad_diag1 = F.conv2d(image, sobel_diag1, padding=1, groups=3)  # (B, 3, H, W)
    grad_diag2 = F.conv2d(image, sobel_diag2, padding=1, groups=3)  # (B, 3, H, W)

    return grad_diag1, grad_diag2

In [ ]:
import torch
import kornia as ki
from kornia.filters import Canny

def compute_canny(images):
    mean = torch.tensor([0.4914, 0.4822, 0.4465], device=images.device).view(1, 3, 1, 1)
    std = torch.tensor([0.2023, 0.1994, 0.2010], device=images.device).view(1, 3, 1, 1)

    imagelle = images * std + mean  # de-normalizza
    img_gray = ki.color.rgb_to_grayscale(imagelle)

    # Usa la classe Canny invece della funzione
    canny = Canny(low_threshold=0.1, high_threshold=0.3, kernel_size=(5, 5)).to(images.device)
    edges, _ = canny(img_gray)

    return edges

In [ ]:
def get_magnitude_mv(images, mv_dimension):
    magnitude = compute_canny(images)
    magnitude = magnitude.squeeze(1)

    batch_size, channels, height, width = images.shape
    mv = torch.zeros(batch_size, height, width, mv_dimension).to(device)

    if mv_dimension == 16:
      mv[..., 0] = magnitude

    return mv

In [ ]:
def get_hessian_mv(images, sobel_x, sobel_y, grad_x, grad_y, mv_dimension):
    hessian = compute_hessian_determinant(images, sobel_x, sobel_y, grad_x, grad_y)

    batch_size, channels, height, width = images.shape
    mv = torch.zeros(batch_size, height, width, global_var['ga_dimension']).to(device)

    if mv_dimension == 16:
      mv[..., 0] = hessian

    return mv

In [ ]:
def compute_hessian_determinant(image, sobel_x, sobel_y, g_x, g_y):
    """ Calcola il determinante dell'Hessiano per un batch di immagini (B, C, H, W). """
    B, C, H, W = image.shape  # (64, 3, 32, 32)

    # Derivate seconde (Hessiano)
    Hxx = F.conv2d(g_x, sobel_x, padding=1, groups=C)  # ∂²f/∂x²
    Hyy = F.conv2d(g_y, sobel_y, padding=1, groups=C)  # ∂²f/∂y²
    Hxy = F.conv2d(g_x, sobel_y, padding=1, groups=C)  # ∂²f/∂x∂y

    # Determinante dell'Hessiano per ogni pixel e canale
    det_H = Hxx * Hyy - Hxy ** 2  # Shape (B, C, H, W)

    det_H_combined = det_H.sum(dim=1) / C

    return det_H_combined

In [ ]:
def get_pos_color_mv(pos, mv_dimension):
    """
    Embed the (n_items,3) position vector in geometric algebra

    Args:
        pos (torch.Tensor): Input position tensor.

    Returns:
        (torch.Tensor): Embed of the position vector in geometric algebra
    """
    batch_size, channels, height, width = pos.shape
    mv = torch.zeros(batch_size, height, width, mv_dimension).to(device)

    if mv_dimension == 16:
      mv[..., 11] = pos[:, 0, :, :] # x-coordinate of pos
      mv[..., 12] = pos[:, 1, :, :] # y-coordinate of pos
      mv[..., 13] = pos[:, 2, :, :]# z-coordinate of pos
      mv[..., 14] = 1 # homogeneous coordinates

    return mv

In [ ]:
def get_gradient_mv(gradient, shift, mv_dimension):
    """
    Embed the (n_items,2) gradient vector in geometric algebra

    Args:
        gradient (torch.Tensor): Input gradient tensor.

    Returns:
        (torch.Tensor): Embed of the gradient vector in geometric algebra
    """

    batch_size, channels, height, width = gradient.shape
    mv = torch.zeros(batch_size, height, width, mv_dimension).to(device)

    mv[..., 5] = shift[:, 0, :, :]
    mv[..., 6] = shift[:, 1, :, :]
    mv[..., 7] = shift[:, 2, :, :]
    mv[..., 8] = gradient[:, 0, :, :]
    mv[..., 9] = gradient[:, 1, :, :]
    mv[..., 10] = gradient[:, 2, :, :]

    return mv

In [ ]:
def get_pos_mv(pos):
    """
    Embed the (n_items,3) position vector in geometric algebra

    Args:
        pos (torch.Tensor): Input position tensor.

    Returns:
        (torch.Tensor): Embed of the position vector in geometric algebra
    """
    n_items, dimension = pos.shape
    # The reshape is necessary so that columns related to x,y,z
    # coordinates can be taken separately
    reshaped_pos = pos.reshape(1,n_items,1,dimension)

    mv_shape = reshaped_pos.shape[:3]
    mv = torch.zeros(*mv_shape, global_var['ga_dimension'])

    mv[..., 14] = 1 # homogeneous coordinates
    mv[..., 11] = reshaped_pos[..., 0] # x-coordinate of pos
    mv[..., 12] = reshaped_pos[..., 1] # y-coordinate of pos
    mv[..., 13] = reshaped_pos[..., 2] # z-coordinate of pos

    return mv

In [ ]:
def get_face_mv(face):
    """
    Embed the (n_items,3) faces vector in geometric algebra

    Args:
        face (torch.Tensor): Input faces tensor.

    Returns:
        (torch.Tensor): Embed of the faces vector in geometric algebra
    """

    n_items, dimension = face.shape
    reshaped_face = face.reshape(1,n_items,1,dimension)

    mv_shape = reshaped_face.shape[:3]
    mv = torch.zeros(*mv_shape, global_var['ga_dimension'])

    mv[..., 2] = reshaped_face[..., 0]
    mv[..., 3] = reshaped_face[..., 1]
    mv[..., 4] = reshaped_face[..., 2]

    return mv

In [ ]:
def get_wss_mv(wss):
    """
    Embed the (n_items,3) wss vector in geometric algebra

    Args:
        pos (torch.Tensor): Input wss tensor.

    Returns:
        (torch.Tensor): Embed of the wss vector in geometric algebra
    """
    n_items, dimension = wss.shape
    reshaped_wss = wss.reshape(1,n_items,1,dimension)

    mv_shape = reshaped_wss.shape[:3]
    mv = torch.zeros(*mv_shape, global_var['ga_dimension'])

    mv[..., 0] = 1 # homogeneous coordinates

    mv[..., 5] = 0.5 * reshaped_wss[..., 0]
    mv[..., 6] = 0.5 * reshaped_wss[..., 1]
    mv[..., 7] = 0.5 * reshaped_wss[..., 2]

    return mv

In [ ]:
def get_inlet_mv(inlet):
    """
    Embed the (n_items,) inlet vector in geometric algebra

    Args:
        pos (torch.Tensor): Input inlet tensor.

    Returns:
        (torch.Tensor): Embed of the inlet vector in geometric algebra
    """
    n_items = inlet.shape[0]
    reshaped_inlet = inlet.reshape(1,n_items,1,1)

    mv_shape = reshaped_inlet.shape[:3]
    mv = torch.zeros(*mv_shape, global_var['ga_dimension'])

    mv[..., 0] = reshaped_inlet[...,0]

    return mv

In [ ]:
def get_pressure_mv(pressure):
    """
    Embed the (n_items,) pressures vector in geometric algebra

    Args:
        pos (torch.Tensor): Input pressures tensor.

    Returns:
        (torch.Tensor): Embed of the pressures vector in geometric algebra
    """
    n_items = pressure.shape[0]
    reshaped_pressure = pressure.reshape(1,n_items,1,1)

    mv_shape = reshaped_pressure.shape[:3]
    mv = torch.zeros(*mv_shape, global_var['ga_dimension'])

    # GATr was found to be more sensitive to widely
    # different values among the data. In terms of range
    # of values, the pressure exceeded the others by several
    # units of magnitude, so it was scaled to its submultiples
    mv[..., 0] = reshaped_pressure[...,0] * 1e-6

    return mv

In [ ]:
def get_plane_reflection_mv(normal,shift):
    """
    Generates the multivector representations for plane reflection and its inverse.

    Parameters:
        normal (torch.Tensor): Normal vector of the plane.
        shift (torch.Tensor): Shift vector of the plane.

    Returns:
        (Tuple[torch.Tensor]): Multivector representations for plane reflection and its inverse.
    """

    inv_shift = -shift

    # [batch,item,channels,mv_dim]
    plane_reflection_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    plane_reflection_mv[..., 1] = shift
    plane_reflection_mv[..., 2] = normal[0]
    plane_reflection_mv[..., 3] = normal[1]
    plane_reflection_mv[..., 4] = normal[2]

    inv_plane_reflection_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    inv_plane_reflection_mv[..., 1] = inv_shift
    inv_plane_reflection_mv[..., 2] = normal[0]
    inv_plane_reflection_mv[..., 3] = normal[1]
    inv_plane_reflection_mv[..., 4] = normal[2]

    return plane_reflection_mv, inv_plane_reflection_mv

In [ ]:
def get_translation_mv(translation):
    """
    Generates the multivector representations for translation and its inverse.

    Parameters:
        translation (List[float]): List containing translation values along x, y, and z axes.

    Returns:
        (Tuple[torch.Tensor]): Multivector representations for translation and its inverse.
    """

    inv_translation = list(map(lambda x: -x, translation))

    # [batch,item,channels,mv_dim]
    translation_mv = torch.zeros(1,1,1,global_var['ga_dimension'])
    translation_mv[..., 0] = 1
    translation_mv[..., 5] = 0.5 * translation[0]
    translation_mv[..., 6] = 0.5 * translation[1]
    translation_mv[..., 7] = 0.5 * translation[2]

    inv_translation_mv = torch.zeros(1,1,1,global_var['ga_dimension'])
    inv_translation_mv[..., 0] = 1
    inv_translation_mv[..., 5] = 0.5 * inv_translation[0]
    inv_translation_mv[..., 6] = 0.5 * inv_translation[1]
    inv_translation_mv[..., 7] = 0.5 * inv_translation[2]

    return translation_mv,inv_translation_mv

In [ ]:
def get_rotation_mv(angle,axis):
    """
    Generates the multivector representations for rotation and its inverse.

    Parameters:
        angle (float): Rotation angle in degrees.
        axis (np.ndarray): Axis of rotation represented as a 3D vector.

    Returns:
        (Tuple[torch.Tensor]): Multivector representations for rotation and its inverse.
    """

    quaternion = quaternion_from_angle_axis(
        alpha = angle,
        axis = axis
    )

    rotation_quaternion = quaternion.quaternion_to_numpy()
    inv_rotation_quaternion = quaternion.conjugate().quaternion_to_numpy()

    # [batch,item,channels,mv_dim]
    rotation_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    rotation_mv[..., 0] = rotation_quaternion[0]
    rotation_mv[..., 8] = rotation_quaternion[1]
    rotation_mv[..., 9] = rotation_quaternion[2]
    rotation_mv[..., 10] = rotation_quaternion[3]

    inv_rotation_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    inv_rotation_mv[..., 0] = inv_rotation_quaternion[0]
    inv_rotation_mv[..., 8] = inv_rotation_quaternion[1]
    inv_rotation_mv[..., 9] = inv_rotation_quaternion[2]
    inv_rotation_mv[..., 10] = inv_rotation_quaternion[3]

    return rotation_mv,inv_rotation_mv

In [ ]:
def get_point_reflection_mv(point):
    """
    Generates the multivector representations for point reflection and its inverse.

    Parameters:
        point (List[float]): Coordinates of the point in 3D space.

    Returns:
        (Tuple[torch.Tensor]): Multivector representations for point reflection and its inverse.
    """

    inv_point = list(map(lambda x: -x, point))
    # [batch,item,channels,mv_dim]
    point_reflection_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    point_reflection_mv[..., 11] = point[0]
    point_reflection_mv[..., 12] = point[1]
    point_reflection_mv[..., 13] = point[2]
    point_reflection_mv[..., 14] = 1

    inv_point_reflection_mv = torch.zeros(1,1,1,global_var['ga_dimension'])

    inv_point_reflection_mv[..., 11] = inv_point[0]
    inv_point_reflection_mv[..., 12] = inv_point[1]
    inv_point_reflection_mv[..., 13] = inv_point[2]
    inv_point_reflection_mv[..., 14] = 1

    return point_reflection_mv,inv_point_reflection_mv

In [ ]:
def embed_data(sample):
    """
    Embed data properties into a multivector

    Args:
        sample (dict): Dataset sample

    Returns:
        (torch.Tensor): Data Sample embedded in a multivector
    """
    # 'pos' property modeled as a point
    cropped_pos = sample['pos'][:global_var['data_crop']]
    input_pos = torch.tensor(np.array(cropped_pos))
    pos_mv = get_pos_mv(input_pos)

    # 'face' property modeled as a plane
    cropped_face = sample['face'][:global_var['data_crop']]
    input_face = torch.tensor(np.array(cropped_face))
    face_mv = get_face_mv(input_face)

    # 'wss' property modeled as a translation
    cropped_wss = sample['wss'][:global_var['data_crop']]
    input_wss = torch.tensor(np.array(cropped_wss))
    wss_mv = get_wss_mv(input_wss)

    # 'pressure' modeled as a scalar
    cropped_pressure = sample['pressure'][:global_var['data_crop']]
    input_pressure = torch.tensor(np.array(cropped_pressure))
    pressure_mv = get_pressure_mv(input_pressure)

    # (batch,n_items,sample_features,ga_dimension)
    total_mvs = torch.cat([pos_mv, face_mv, wss_mv, pressure_mv],dim = 2)

    return total_mvs

## Blade
The blade is a multivector $<x>_k$ operator. Applied to a multivector $x$, it returns $x$ with only the components of grade $k$.

In [ ]:
def blade_operator():
    """
    Generates a blade operator matrix for the geometric algebra

    Args:
        None

    Returns:
        (torch.Tensor): Blade operator matrix.
    """

    mv_dimension = global_var['ga_dimension']
    blade_shape = (mv_dimension, mv_dimension)

    coordinates = []
    start = 0
    for length in global_var['grade_components']:
        coordinates.append(list(range(start, start + length)))
        start += length

    coord_permutations = global_var['ga_permutation']
    blade_mask = []

    w_dimension = len(global_var['grade_components'])
    for k_grade in range(w_dimension):
        w_blade = torch.zeros(blade_shape)
        for coordinate in coordinates[k_grade]:
            w_blade[coordinate, coordinate] = 1.0
        blade_mask.append(w_blade.unsqueeze(0))

    v_dimension = len(global_var['grade_components']) - 1
    for k_grade in range(v_dimension):
        v_blade = torch.zeros(blade_shape)
        for coord_to,coord_from in coord_permutations[k_grade]:
            if coord_from < mv_dimension and coord_to < mv_dimension:
                v_blade[coord_from, coord_to] = 1.0
            else:
                print(f"Warning: Indici fuori limiti: coord_from={coord_from}, coord_to={coord_to}")
        blade_mask.append(v_blade.unsqueeze(0))

    blade_operator = torch.cat(blade_mask,dim = 0)

    return blade_operator

## Reverse
The reverse $\widetilde{x}$ is an operator to be applied to a multivector that changes the signs of its bivectors and trivectors. In applications, it can be reassimilated to a transposition.

In [ ]:
def get_coordinates_range():
    """
    Get the ranges of coordinates for each grade based on the configuration in `global_var`.

    Args:
        None

    Returns:
        (List[List[int]]): List of coordinate ranges for each grade.
    """
    grade_components = global_var['grade_components']
    coordinates_range = []

    for grade in range(len(grade_components)):
        start_idx = sum(grade_components[:grade])
        end_idx = sum(grade_components[:grade + 1]) - 1
        coordinate_range = [start_idx,end_idx]
        coordinates_range.append(coordinate_range)

    return coordinates_range

In [ ]:
def reverse_operator():
    """
    Generate a reverse operator for the given multivector configuration.

    Args:
        None

    Returns:
        (torch.Tensor): Reverse operator for the multivector space.
    """

    reverse_operator = torch.ones(global_var['ga_dimension'])
    *_, bivector_range, trivector_range, _ = get_coordinates_range()

    reverse_range = list(
        range(
            bivector_range[0],
            trivector_range[-1] + 1
         )
     )

    reverse_operator[reverse_range] = -1

    return reverse_operator.to(device)

## Dual
The reverse $x^*$ is an operator that generalize the blade. It changes the signs of the even mutlivector components.

In [ ]:
def dual_operators():
    """
    Constructs dual operators, including the indices for sign flipping and the sign values.

    Args:
        None

    Returns:
        (Tuple[List[int], torch.Tensor]): Tuple containing a list of indices to mask to perfom dual and
                                          the vector to perfom it. This masked approach make faster the
                                          dual computation when applied to a multivector
    """

    coords_range = get_coordinates_range()

    dual_sign_idxs = [2,4,6,9,12,14]
    dual_signs = [
        1 if i not in dual_sign_idxs else -1
        for i in range(global_var['ga_dimension'])
    ][::-1]
    dual_signs = torch.tensor(dual_signs)

    dual_flip = list(
        range(
            coords_range[-1][0],coords_range[0][0]-1,-1
         )
     )

    return dual_flip, dual_signs.to(device)

## Geometric product

The geometric product $xy$ is a particular multiplication that has two multivectors as arguments. This product respects some properties:

* The product of the same argument must be equal to its norm squared: $vv = ||v||^2$.
* The product between bases is antisymmetric: $e_ie_j= -e_je_i $ $∀i\neq j$.
* The product of the space basis squared is one: $e_i^2= 1$ $ ∀i\neq 0$.
* The product of the projective basis squared is zero: $e_0^2= 0$.

To realize this particular product, Einstein's summation convention is very convenient. This convention provides a concise way to express complex multidimensional tensor operations.

In order to obtain the properties correctly, after some unsuccessful attempts or otherwise not accurate and consistent for the desired results, we have exploited a guiding matrix proposed by [1,3] to correctly and efficiently realize the product operations (geometric and non-geometric) of the geometric algebra in question.


In [ ]:
def get_guidance_matrix_8():
    dim = 8
    guidance_matrix_8 = torch.zeros((dim, dim, dim))

    # Indici per gli elementi base
    scalar, e0, e1, e2, e01, e02, e12, e012 = range(8)

    guidance_matrix_8[scalar, scalar, scalar] = 1
    guidance_matrix_8[scalar, e0, e0] = 0
    guidance_matrix_8[scalar, e1, e1] = 1
    guidance_matrix_8[scalar, e2, e2] = 1
    guidance_matrix_8[scalar, e01, e01] = 0
    guidance_matrix_8[scalar, e02, e02] = 0
    guidance_matrix_8[scalar, e12, e12] = -1
    guidance_matrix_8[scalar, e012, e012] = 0

    guidance_matrix_8[e0, scalar, e0] = 1
    guidance_matrix_8[e0, e0, scalar] = 1
    guidance_matrix_8[e0, e1, e01] = -1
    guidance_matrix_8[e0, e2, e02] = -1
    guidance_matrix_8[e0, e01, e1] = 1
    guidance_matrix_8[e0, e02, e2] = 1
    guidance_matrix_8[e0, e12, e012] = -1

    guidance_matrix_8[e1, scalar, e1] = 1
    guidance_matrix_8[e1, e0, e01] = 0
    guidance_matrix_8[e1, e1, scalar] = 1
    guidance_matrix_8[e1, e2, e12] = -1
    guidance_matrix_8[e1, e01, e0] = 0
    guidance_matrix_8[e1, e02, e012] = 0
    guidance_matrix_8[e1, e12, e2] = 1

    guidance_matrix_8[e2, scalar, e2] = 1
    guidance_matrix_8[e2, e0, e02] = 0
    guidance_matrix_8[e2, e1, e12] = 1
    guidance_matrix_8[e2, e2, scalar] = 1
    guidance_matrix_8[e2, e01, e012] = 0
    guidance_matrix_8[e2, e02, e0] = 0
    guidance_matrix_8[e2, e12, e1] = -1
    guidance_matrix_8[e2, e012, e01] = 0

    guidance_matrix_8[e01, scalar, e01] = 1
    guidance_matrix_8[e01, e0, e1] = 1
    guidance_matrix_8[e01, e1, e0] = -1
    guidance_matrix_8[e01, e2, e012] = 1
    guidance_matrix_8[e01, e01, scalar] = 1
    guidance_matrix_8[e01, e02, e12] = -1
    guidance_matrix_8[e01, e12, e02] = 1
    guidance_matrix_8[e01, e012, e2] = 1

    guidance_matrix_8[e02, scalar, e02] = 1
    guidance_matrix_8[e02, e0, e2] = 1
    guidance_matrix_8[e02, e1, e012] = -1
    guidance_matrix_8[e02, e2, e0] = -1
    guidance_matrix_8[e02, e01, e12] = 1
    guidance_matrix_8[e02, e02, scalar] = 1
    guidance_matrix_8[e02, e12, e01] = -1
    guidance_matrix_8[e02, e012, e1] = -1

    guidance_matrix_8[e12, scalar, e12] = 1
    guidance_matrix_8[e12, e0, e012] = 0
    guidance_matrix_8[e12, e1, e2] = 1
    guidance_matrix_8[e12, e2, e1] = -1
    guidance_matrix_8[e12, e01, e02] = 0
    guidance_matrix_8[e12, e02, e01] = 0
    guidance_matrix_8[e12, e12, scalar] = 1
    guidance_matrix_8[e12, e012, e0] = 0

    guidance_matrix_8[e012, scalar, e012] = 1
    guidance_matrix_8[e012, e0, e12] = 1
    guidance_matrix_8[e012, e1, e02] = -1
    guidance_matrix_8[e012, e2, e01] = 1
    guidance_matrix_8[e012, e01, e2] = 1
    guidance_matrix_8[e012, e02, e1] = -1
    guidance_matrix_8[e012, e12, e0] = 1
    guidance_matrix_8[e012, e012, scalar] = 1

    return guidance_matrix_8

In [ ]:
def get_guidance_matrix():
    """
    Fetches and loads the guidance matrix from a specified URL,
    saving it locally if not already present.

    Returns:
        (torch.Tensor): A tensor representing the guidance matrix.
    """
    if not os.path.exists("guidance_matrix.pt"):
        matrix_url = global_var['prefix'] + global_var['guidance_id']
        matrix_file = wget.download(matrix_url)

    if global_var['ga_dimension'] == 16:
        guidance_matrix = product_basis = torch.load("guidance_matrix.pt", weights_only=True)
    else:
        guidance_matrix = get_guidance_matrix_8()
    guidance_matrix = guidance_matrix.to(torch.float32)
    guidance_matrix = guidance_matrix.to_dense()

    return guidance_matrix

In [ ]:
def geometric_product(x,y):
    """
    Computes the geometric product of two multivectors using the guidance matrix.

    Args:
        x (torch.Tensor): the first multivector
        y (torch.Tensor): the second multivector

    Returns:
        (torch.Tensor): multivector result of the geometric product
    """

    guidance_matrix = get_guidance_matrix().to(device)

    geom_prod = torch.einsum(
        "i j k, ... j, ... k -> ... i",
        guidance_matrix,
        x,
        y
     )

    return geom_prod

## Inner product
The inner product $<x,y>$ is a generalized version of the well-known scalar product, applied to multivectors. Conceptually, it indicates the similarity between two multivectors.

In the practical context, to compute the inner product, we exploit its form $<x,y> = <\tilde{x}y>_0$, thus computing the blade of degree 0 related to the geometric product between the inverse of the first operand and the second operand.

This type of approach is correct but may become very slow for tensors with enough elements. Thus [1], as done for the dual operator, proposes a mask approach that isolates only the components that will actually be affected by the geometric product, thus achieving the same result but in less time.

In [ ]:
def inner_product(x,y):
    """
    Computes the inner product of two multivectors.

    Args:
        x (torch.Tensor): the first multivector
        y (torch.Tensor): the second multivector

    Returns:
        (torch.Tensor): result of the inner product
    """

    reverse_x = reverse_operator() * x
    geom_x_y = geometric_product(reverse_x,y)
    inner_product = geom_x_y[..., [0]]

    return inner_product

In [ ]:
def faster_inner_product(x,y):
    """
    Computes the inner product of two multivectors using a faster method.

    Args:
        x (torch.Tensor): the first multivector
        y (torch.Tensor): the second multivector

    Returns:
        (torch.Tensor): scalar result of the inner product
    """
    guidance_matrix = get_guidance_matrix().to(device)
    reverse_op = reverse_operator()

    inner_product_mask = (
        torch.diag(guidance_matrix[0]) * reverse_op
    ).bool()

    x = x[..., inner_product_mask]
    y = y[..., inner_product_mask]

    # questo è un "geom prod2 modificato"
    inner_product = torch.einsum(
        "... i, ... i -> ...", x, y
    ).unsqueeze(-1)


    return inner_product

## Outer product
The outer product $x \wedge y$ represents another particular product of algebraic geometry, conceptually close to the well-known vector product. This product follows the thread of the geometric product, but with a slightly different product guide, again proposed and implemented by [1].

In [ ]:
def get_outer_matrix_8():
    dim = 8
    outer_matrix_8 = torch.zeros((dim, dim, dim))

    # Indici per gli elementi base
    scalar, e0, e1, e2, e01, e02, e12, e012 = range(8)

    outer_matrix_8[scalar, scalar, scalar] = 1

    outer_matrix_8[e0, scalar, e0] = 1
    outer_matrix_8[e0, e0, scalar] = 1

    outer_matrix_8[e1, scalar, e1] = 1
    outer_matrix_8[e1, e1, scalar] = 1

    outer_matrix_8[e2, scalar, e2] = 1
    outer_matrix_8[e2, e2, scalar] = 1

    outer_matrix_8[e01, scalar, e01] = 1
    outer_matrix_8[e01, e0, e1] = 1
    outer_matrix_8[e01, e1, e0] = -1
    outer_matrix_8[e01, e01, scalar] = 1

    outer_matrix_8[e02, scalar, e02] = 1
    outer_matrix_8[e02, e0, e2] = 1
    outer_matrix_8[e02, e2, e0] = -1
    outer_matrix_8[e02, e02, scalar] = 1

    outer_matrix_8[e12, scalar, e12] = 1
    outer_matrix_8[e12, e1, e2] = 1
    outer_matrix_8[e12, e2, e1] = -1
    outer_matrix_8[e12, e12, scalar] = 1

    outer_matrix_8[e012, scalar, e012] = 1
    outer_matrix_8[e012, e0, e12] = 1
    outer_matrix_8[e012, e1, e02] = -1
    outer_matrix_8[e012, e2, e01] = 1
    outer_matrix_8[e012, e01, e2] = 1
    outer_matrix_8[e012, e02, e1] = -1
    outer_matrix_8[e012, e12, e0] = 1
    outer_matrix_8[e012, e012, scalar] = 1

    return outer_matrix_8

In [ ]:
def get_outer_matrix():
    """
    Fetches and loads the outer product guidance matrix
    from a specified URL,  saving it locally if not already present.

    Returns:
        (torch.Tensor): A tensor representing the outer guidance matrix.
    """

    if not os.path.exists("outer_guidance.pt"):
        matrix_url = global_var['prefix'] + global_var['outer_id']
        matrix_file = wget.download(matrix_url)

    if global_var['ga_dimension'] == 16:
        outer_matrix = torch.load("outer_guidance.pt")
    else:
        outer_matrix = get_outer_matrix_8()
    outer_matrix = outer_matrix.to(torch.float32)
    outer_matrix = outer_matrix.to_dense()

    return outer_matrix

In [ ]:
def outer_product(x,y):
    """
    Computes the outer product of two multivectors using the outer guidance matrix.

    Args:
        x (torch.Tensor): the first multivector
        y (torch.Tensor): the second multivector

    Returns:
        (torch.Tensor): multivector result of the outer product
    """

    outer_matrix = get_outer_matrix().to(device)

    outputs = torch.einsum("i j k, ... j, ... k -> ... i", outer_matrix, x, y)
    return outputs

## Join
The join $x,y = (x^* \wedge y^*)^*$ is an operation that, conceptually is like placing an object of geometric algebra between the two elements $x$ and $y$, as a "line" connecting them. In project algebraic geometry, however, the intrinsic nature of the join is nonequivariant. The authors of [1] propose a solution by multiplying the result of the join by a reference, i.e., the average of the first multivector entering the GATr. This technique is demonstrated in the paper [1] and discussed in this [video](https://www.youtube.com/watch?v=YbTSgnzg26Q&t=2505s).


In [ ]:
def join(x,y,ref):
    """
    Computes the join of two multivectors relative to a reference multivector.

    Args:
        x (torch.Tensor): the first multivector
        y (torch.Tensor): the second multivector
        ref (torch.Tensor): the reference multivector, mean over the batch
                            and channels of the entering GATr multivector

    Returns:
        (torch.Tensor): multivector result of the join operation
    """

    dual_flip,dual_signs = dual_operators()
    dual_x = dual_signs * x[...,dual_flip]
    dual_y = dual_signs * y[...,dual_flip]

    outer_prod = outer_product(dual_x, dual_y)
    classic_join = dual_signs * outer_prod[...,dual_flip]
    equi_join = ref[..., [global_var['ga_dimension'] - 1]] * classic_join

    return equi_join

## Grade involution
The grade involution $\hat{x}$ is an operator that slightly modifies the blade operator. In fact, it changes the signs of the odd mutlivector components.

In [ ]:
def grade_involution(mv):
    """
    Applies the grade involution to a multivector, flipping the signs of odd-graded components.

    Args:
        mv (torch.Tensor): the input multivector

    Returns:
        (torch.Tensor): multivector result of the grade involution
        (list): indices of flipped signs during the involution
    """


    odd_grades = get_coordinates_range()[1::2]
    flip_signs = []

    for grade_range in odd_grades:
        flip_signs += list(range(grade_range[0],grade_range[1] + 1))

    involution_signs = [-1 if i in flip_signs else 1 for i in range(global_var['ga_dimension'])][::-1]
    involution_signs = torch.tensor(involution_signs).to(device)

    involuted_mv = involution_signs * mv

    return involuted_mv, flip_signs

## Sandwich product
The sandwich product $\rho_u(x)$ allows the operator $u$ to be applied to the multivector $x$. The application of the latter is done by a geometric product between $u$, $x$ and the inverse operator $u^{-1}$. In particular, if the operator has elements of odd degree, the multivector being applied must undergo grade involution, before performing the above products.

<br>
<center>
$
\rho_u(x) = \left\{\begin{matrix}
 uxu^{-1}& \text{if}\enspace u \enspace \text{even}\\
 u\hat{x}u^{-1}& \text{if}\enspace u \enspace \text{odd}\
\end{matrix}\right.
$
</center>
<br>

In [ ]:
def sandwich_product(mv,u,inv_u):
    """
    Computes the sandwich product of a multivector and a matrix.

    Args:
        mv (torch.Tensor): the input multivector
        u (torch.Tensor): the matrix for the sandwich product

    Returns:
        (torch.Tensor): a torch tensor representing the result
                        of the sandwich product
    """
    first_geom_product = geometric_product(u,mv)
    output = geometric_product(first_geom_product,inv_u)

    return output

## Equivariant check
The particularity of the neural network of the project, the GATr proposed by [1] is its ability to be equivariant with respect to the operators of E(3), the rigid transformations of 3D space: rotations, translations and reflections. This particularity is an important element to achieve, as it differs from other Transformer-type architectures and will be able to be an important term of comparison. Indeed, as noted in the EDA section, the binary classification task, could prove to be quite simple, especially for very powerful architectures such as those that will be presented. So the peculiarity of having an equivariant approach would certainly be a point in favor of the GATr architecture.

To check that indeed this property is respected, the equation to check is as follows.

<br>
<center>
$f(\rho_u(x)) = \rho_u(f(x))$
</center>
<br>

This check will be applied for each layer $f$ and for each operator in the table at the beginning of the section.
However, there is to specify how due to some imperfections, such as numerical approximations and too deep details in the geometric algebra, it is very unlikely that the two sides of the equation, passed in complex layers, and relatively imperfect as mentioned above, such as those of GATr, will be equal. Perhaps they may be very similar, with very little difference in terms of elements, but the latter would give a negative result if we applied a classical control such as torch.equal. Hence the design choice to analyze al distance between the two sides of the equation presented above, considering ourselves satisfied if the distance between these two values is very low.

In [ ]:
def difference_distance(first_side,second_side):
    """
    Computes the difference distance between two sides represented by tensors.

    Args:
        first_side (torch.Tensor): the first side represented as a tensor
        second_side (torch.Tensor): the second side represented as a tensor

    Returns:
        (float): the Euclidean norm of the difference between the two sides
    """

    first_side = first_side.cpu().detach().numpy()
    second_side = second_side.cpu().detach().numpy()
    return np.linalg.norm(first_side - second_side)

In [ ]:
def equivariant_check(mv,operators,layer,ref=None):
    """
    Checks the equivariance of a GATr layer with respect to a multivector.

    Args:
        mv (torch.Tensor): the input multivector
        operators (List): list containing the direct and inverse operators
        layer (torch.nn.Module): the GATr layer
        ref (torch.Tensor, optional): reference multivector for join

    Returns:
        (List): contains the two equivariant equation sides and normalized difference
                distance between them to assess equivariance
    """

    batch, items, channels, _ = mv.shape
    denominator = batch * items * channels
    equicheck_in = mv.to(device)
    operator = operators[0].to(device)
    inv_operator = operators[1].to(device)
    involuted_mv, flipped_signs = grade_involution(equicheck_in)
    layer = layer.to(device)

    if torch.any(operator[...,flipped_signs] != 0):
        equicheck_in = involuted_mv.to(device)

    if ref is not None:
        first_side = sandwich_product(
            mv = layer(equicheck_in,ref),
            u = operator,
            inv_u = inv_operator
        )
        second_side = layer(
            sandwich_product(
                mv = equicheck_in,
                u = operator,
                inv_u = inv_operator
                ),
            ref
        )
    else:
        first_side = sandwich_product(
            mv = layer(equicheck_in),
            u = operator,
            inv_u = inv_operator
        )
        second_side = layer(
            sandwich_product(
                mv = equicheck_in,
                u = operator,
                inv_u = inv_operator
            )
        )

    output = difference_distance(first_side,second_side)
    output /= denominator

    return output

# Dataset

In [ ]:
from scipy.io import loadmat
import numpy as np
import h5py

'''
# Carica l'immagine iperspettrale
data = loadmat('/content/Indian_pines_corrected.mat')['indian_pines_corrected']  # shape: (145, 145, 200)

# Carica le etichette
gt = loadmat('/content/Indian_pines_gt.mat')['indian_pines_gt']  # shape: (145, 145)

data_class_names = [
    "Alfalfa", "Corn-notill", "Corn-mintill", "Corn", "Grass-pasture", "Grass-trees",
    "Grass-pasture-mowed", "Hay-windrowed", "Oats", "Soybean-notill", "Soybean-mintill", "Soybean-clean", "Wheat",
    "Woods", "Buildings-Grass-Trees-Drives", "Stone-Steel-Towers"
                    ]

pre_name = "ip"
n_classes = 16
ten_per = True
global_ten_per = True
'''

# Carica l'immagine iperspettrale
ksc_data = loadmat('/content/KSC_corrected.mat')
data = ksc_data['KSC']

# Carica le etichette
ksc_labels = loadmat('/content/KSC_gt.mat')
gt = ksc_labels['KSC_gt']

data_class_names = ["Scrub", "Willow swamp", "Cabbage palm hammock", "Cabbage palm/oak hammock",
                    "Slash pine", "Oak/broadleaf hammock", "Hardwood swamp", "Graminoid marsh",
                    "Spartina marsh", "Cattail marsh", "Salt marsh", "Mud flats", "Water"]

ten_per = True
global_ten_per = True
pre_name = "ksc"
n_classes = 13


'''
ksc_data = loadmat('/content/Salinas_corrected.mat')
data = ksc_data['salinas_corrected']

# Carica le etichette
ksc_labels = loadmat('/content/Salinas_gt.mat')
gt = ksc_labels['salinas_gt']

data_class_names = [
    "Brocoli_green_weeds_1", "Brocoli_green_weeds_2", "Fallow", "Fallow_rough_plow",
    "Fallow_smooth", "Stubble", "Celery", "Grapes_untrained", "Soil_vinyard_develop",
    "Corn_senesced_green_weeds", "Lettuce_romaine_4wk", "Lettuce_romaine_5wk", "Lettuce_romaine_6wk",
    "Lettuce_romaine_7wk", "Vinyard_untrained", "Vinyard_vertical_trellis"]

pre_name = "salinas"
n_classes = 16
ten_per = False
n_samples = 100
'''
'''
ksc_data = loadmat('/content/PaviaU.mat')
data = ksc_data['paviaU']
# Carica le etichette
ksc_labels = loadmat('/content/PaviaU_gt.mat')
gt = ksc_labels['paviaU_gt']

data_class_names = ["Asphalt", "Meadows", "Gravel", "Trees", "Painted metal sheets	", "Bare Soil	", "Bitumen", "Self-Blocking Bricks", "Shadows"]


pre_name = "pavia"
n_classes = 9
ten_per = False
n_samples = 100
'''

# Carica le etichette

In [ ]:
pip install fetch-houston2013

In [ ]:
from fetch_houston2013 import fetch_houston2013, fetch_muufl, fetch_trento, split_spmatrix
import math
'''
# For Houston 2013
data, dsm, train_label, test_label, info = fetch_houston2013()
data_class_names = ["Healthygrass", "Stressedgrass", "Syntheticgrass", "Trees", "Soil	", "Water", "Residential", "Commercial",
                    "Road", "Highway", "Railway", "Parking lot 1", "Parking lot 2", "Tennis court", "Running track"]
data = data.transpose(1, 2, 0)  # shape: (canali, altezza, larghezza)
pre_name = "houston_13"
n_classes = 15
ten_per = False
n_samples = 100

dsm_tensor = torch.from_numpy(dsm).float()
dsm_mean = dsm_tensor.mean()
dsm_std = dsm_tensor.std()
dsm_norm = (dsm_tensor - dsm_mean) / dsm_std

train_dense = train_label.todense()
test_dense = test_label.todense()

print("HSI shape:", data.shape)          # dovrebbe essere (H, W, C)
print("DSM shape:", dsm.shape)           # dovrebbe essere (1 o 2, H, W)
print("Truth shape:", train_dense.shape) # dovrebbe essere (H, W)
print("Truth shape:", test_dense.shape)
print("Unique labels in truth:", np.unique(train_dense))
for c in np.unique(train_dense):
    if c == 0:
        continue
    n_total = np.sum(train_dense == c)
    print(f"Classe {c}: {n_total} pixel")
print("Train labels min/max:", train_dense.min(), train_dense.max())
print("Test labels min/max:", test_dense.min(), test_dense.max())


'''
'''
# For Muufl
data, dsm, truth, info = fetch_muufl()
data = data.transpose(1, 2, 0)  # shape: (canali, altezza, larghezza)
dsm = dsm[0:1, :, :]
#dsm = dsm.transpose(1, 2, 0)
pre_name = "muufl"
data_class_names = ["trees","grass_pure","grass_groundsurface","dirt_and_sand","road_materials","water","shadow_building","buildings","sidewalk","yellowcurb","cloth_panels"]
n_classes = 11
ten_per = False
n_samples = 20

dsm_tensor = torch.from_numpy(dsm).float()
dsm_mean = dsm_tensor.mean()
dsm_std = dsm_tensor.std()
dsm_norm = (dsm_tensor - dsm_mean) / dsm_std

# Converti in array denso se è sparsa
truth_dense = truth.toarray() if hasattr(truth, "toarray") else truth

# Trova le classi presenti (escludi lo 0 = background)
classes = np.unique(truth_dense)
print(classes)
classes = classes[classes > 0]


# Inizializza le maschere
train_label = np.zeros_like(truth_dense)
test_label = np.zeros_like(truth_dense)

np.random.seed(42)  # riproducibilità

for c in classes:
    # Trova tutti i pixel di questa classe
    coords = np.argwhere(truth_dense == c)
    n_total = len(coords)

    # Calcola il numero di pixel per il training (arrotondato per eccesso)
    n_train = math.ceil(0.1 * n_total)

    # Scegli casualmente i pixel di training
    train_idx = np.random.choice(n_total, n_train, replace=False)
    train_coords = coords[train_idx]

    # Assegna ai set
    for (i, j) in train_coords:
        train_label[i, j] = c
    for (i, j) in coords:
        if train_label[i, j] == 0:
            test_label[i, j] = c

# Conversione a torch.Tensor
train_dense = train_label
test_dense = test_label


print("-------------------")

print("HSI shape:", data.shape)          # dovrebbe essere (H, W, C)
print("DSM shape:", dsm.shape)           # dovrebbe essere (1 o 2, H, W)
print("Truth shape:", train_dense.shape) # dovrebbe essere (H, W)
print("Truth shape:", test_dense.shape)
print("Unique labels in truth:", np.unique(train_dense))
for c in np.unique(train_dense):
    if c == 0:
        continue
    n_total = np.sum(train_dense == c)
    print(f"Classe {c}: {n_total} pixel")
print("Train labels min/max:", train_dense.min(), train_dense.max())
print("Test labels min/max:", test_dense.min(), test_dense.max())
'''
'''
# For Trento
data, dsm, truth, info = fetch_trento()
data = data.transpose(1, 2, 0)
dsm = dsm[0:1, :, :]
data_class_names = ["Appletrees", "Buildings", "Ground", "Woods", "Vineyard", "Roads"]
pre_name = "trento"
n_classes = 6
ten_per = False
n_samples = 20
dsm_tensor = torch.from_numpy(dsm).float()
dsm_mean = dsm_tensor.mean()
dsm_std = dsm_tensor.std()
dsm_norm = (dsm_tensor - dsm_mean) / dsm_std # Carica gli indici del train set
train_coords = np.load("/content/all_train_indices.npy") # Nx2 # Dimensioni immagine
H, W = truth.shape # o usa l’altezza/larghezza della tua immagine

# Inizializza maschere
train_label = np.zeros((H, W), dtype=np.uint8)
test_label = np.zeros((H, W), dtype=np.uint8)

# Imposta tutti i pixel del train set a 1 (o un qualsiasi valore >0)
r, c = train_coords[:,0], train_coords[:,1]

truth_dense = truth.toarray() if hasattr(truth, "toarray") else truth

# train_label contiene il valore della classe originale
train_label[r, c] = truth_dense[r, c]  # invece di 1
# test_label = tutti gli altri pixel
test_label = np.where(train_label == 0, truth_dense, 0)


# Se vuoi trasformare in torch.Tensor
train_dense = torch.from_numpy(train_label)
test_dense = torch.from_numpy(test_label)
print(f"Train set: {train_dense.sum()} pixel")
print(f"Test set: {test_dense.sum()} pixel")
'''


In [ ]:
## Utilizzando fibonacci sphere
import math

def label_to_multivector(samples=10):

    labels = []
    phi = math.pi * (math.sqrt(5.) - 1.)  # golden angle in radians

    for i in range(1, samples + 1):
        y = 1 - (i / float(samples - 1 + 2)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius

        e0, e1, e2, e3 = torch.zeros(16, device=device), torch.zeros(16, device=device), torch.zeros(16, device=device), torch.zeros(16, device=device)

        e0[1] = 0
        e1[2] = x
        e2[3] = y
        e3[4] = z

        e0e1 = geometric_product(e0, e1)
        e0e2 = geometric_product(e0, e2)
        e0e3 = geometric_product(e0, e3)

        e1e2 = geometric_product(e1, e2)
        e1e3 = geometric_product(e1, e3)
        e2e3 = geometric_product(e2, e3)

        e0e1e2 = geometric_product(e0e1, e2)
        e0e1e3 = geometric_product(e0e1, e3)
        e0e2e3 = geometric_product(e0e2, e3)
        e1e2e3 = geometric_product(e1e2, e3)
        e0e1e2e3 = geometric_product(e0e1e2, e3)

        first = torch.zeros(16, device = device)

        new_label = sum([first, e0, e1, e2, e3, e0e1, e0e2, e0e3, e1e2, e1e3, e2e3, e0e1e2, e0e2e3, e1e2e3, e0e1e2e3])

        #new_label = sum([e0, e1, e2, e3])

        labels.append(new_label)

    return torch.stack(labels)

In [ ]:
multivector_labes = label_to_multivector(n_classes)

# GATR

## Model

### Layers

In [ ]:
class EquiLinearLayer(nn.Module):
    """
    Equivariant Linear Layer.

    Args:
        input_mv_channels (int): number of input channels in the multivector
        hidden_mv_dim (int): number of output channels in the multivector
        blade (torch.Tensor): blade tensor representing the geometric entity
        blade_len (int): length of the blade tensor

    Attributes:
        blade (torch.Tensor): blade tensor representing blade operator
        weights (nn.Parameter): learnable weights for the linear layer

    Methods:
        forward(x): computes the forward pass of the equivariant linear layer
    """

    def __init__(self, blade, blade_len, in_mv_channels, out_mv_channels, dropout_p = 0.0):
        super(EquiLinearLayer,self).__init__()

        self.blade = blade

        self.weights = nn.Parameter(
            torch.rand(out_mv_channels, in_mv_channels, blade_len, device=device)
         )

        self.dropout = nn.Dropout(p=dropout_p).to(device)


    def forward(self, multivectors):
        """
        Parameters
        ----------
            multivectors : torch.Tensor with shape (..., in_mv_channels, 16)
            Input multivectors
            scalars : None or torch.Tensor with shape (..., in_s_channels)
            Optional input scalars

        Returns
        -------
        outputs_mv : torch.Tensor with shape (..., out_mv_channels, 16)
            Output multivectors
        outputs_s : None or torch.Tensor with shape (..., out_s_channels)
            Output scalars, if scalars are provided. Otherwise None.
        """

        multivectors = self.dropout(multivectors)
        outputs_mv = torch.einsum(
            "j i b, b x y, ... i x -> ... j y",
            self.weights,
            self.blade,
            multivectors
        )

        return outputs_mv

In [ ]:
class EquilinearNormLayer(nn.Module):
    """
    Custom layer for normalizing multivectors in an equivariant neural network.

    Args:
        faster (bool): flag indicating whether to use the faster inner product calculation

    Attributes:
        faster (bool): flag indicating whether to use the faster inner product calculation

    Methods:
        forward(x): computes the forward pass to normalize multivectors in an equivariant manner
    """

    def __init__(self, faster = True):
        super(EquilinearNormLayer, self).__init__()
        self.faster = faster

    def forward(self, x):
        if self.faster:
            mv_inner_product = faster_inner_product(x, x)
        else:
            mv_inner_product = inner_product(x, x)

        squared_norms = torch.mean(mv_inner_product, dim=-2, keepdim=True)

        # Rescale inputs
        outputs = x / torch.sqrt(squared_norms)

        return outputs

In [ ]:
def prepare_qkv(q, k, v, n_heads, hidden_dim):
    """
    Prepares the query (q), key (k), and value (v) tensors for the
    attention mechanism in a transformer, with the inner product.

    Args:
        q (torch.Tensor): query tensor
        k (torch.Tensor): key tensor
        v (torch.Tensor): value tensor
        n_heads (int): number of attention heads
        hidden_dim (int): hidden dimension

    Returns:
        (torch.Tensor): modified query tensor
        (torch.Tensor): modified key tensor
        (torch.Tensor): modified value tensor
    """

    q = rearrange(
        q,
        "... items (hidden_dim n_heads) mv -> ... n_heads items hidden_dim mv",
        n_heads = n_heads,
        hidden_dim = hidden_dim
     )

    k = rearrange(
        k,
        "... n_items (hidden_dim 1) mv_dim -> ... 1 n_items hidden_dim mv_dim"
     )

    v = rearrange(
        v,
        "... n_items (hidden_dim 1) mv_dim -> ... 1 n_items hidden_dim mv_dim"
     )

    #q_s = pos_encoding(q_s)
    #k_s = pos_encoding(k_s)

    # Questa parte è una costante???
    guidance_matrix = get_guidance_matrix().to(device)
    reverse_op = reverse_operator()
    inner_product_mask = (torch.diag(guidance_matrix[0]) * reverse_op).bool()

    ranges = get_coordinates_range()
    index_product_idxs = list(range(ranges[0][0],ranges[-1][-1]+1))
    index_product_coordinates = [
        coord for coord,
        keep in zip(index_product_idxs, inner_product_mask) if keep
     ]

    q = rearrange(q[..., index_product_coordinates], "... c x -> ... (c x)")
    k = rearrange(k[..., index_product_coordinates], "... c x -> ... (c x)")
    v = rearrange(v, "... c x -> ... (c x)")

    return q,k,v

In [ ]:
class GeometricAttentionLayer(nn.Module):
    """
    Geometric equivariant attention

    Args:
        mv_channels (int): number of channels in the input multivector
        hidden_dim (int): hidden dimension for attention calculations
        out_channels (int): number of channels in the output multivector
        blade (torch.Tensor): blade tensor representing the geometric entity
        blade_len (int): length of the blade tensor
        n_heads (int): number of attention heads

    Attributes:
        n_heads (int): number of attention heads
        mv_channels (int): number of channels in the input multivector
        hidden_dim (int): hidden dimension for attention calculations
        out_channels (int): number of channels in the output multivector
        q (EquiLinearLayer): layer for queries
        k (EquiLinearLayer): layer for keys
        v (EquiLinearLayer): layer for values
        output_projection (EquiLinearLayer): layer for output projection

    Methods:
        forward(x): computes the forward pass of the geometric attention layer
    """
    def __init__(
        self,mv_channels,hidden_dim,out_channels,blade,blade_len,n_heads
    ):
        super(GeometricAttentionLayer,self).__init__()
        self.n_heads = n_heads
        self.mv_channels = mv_channels
        self.hidden_dim = hidden_dim
        self.out_channels = out_channels

        self.q = EquiLinearLayer(
            in_mv_channels = mv_channels,
            out_mv_channels = hidden_dim * n_heads,
            blade = blade,
            blade_len = blade_len
         )

        self.k = EquiLinearLayer(
            in_mv_channels = mv_channels,
            out_mv_channels = hidden_dim,
            blade = blade,
            blade_len = blade_len
         )

        self.v = EquiLinearLayer(
            in_mv_channels = mv_channels,
            out_mv_channels = hidden_dim,
            blade = blade,
            blade_len = blade_len
         )

        self.output_projection = EquiLinearLayer(
            in_mv_channels = hidden_dim * n_heads,
            out_mv_channels = out_channels,
            blade = blade,
            blade_len = blade_len
         )

    def forward(self, x):
        q_mv = self.q(x)
        k_mv = self.k(x)
        v_mv = self.v(x)

        q,k,v = prepare_qkv(
            q_mv,
            k_mv,
            v_mv,
            self.n_heads,
            self.hidden_dim
         )

        attention = scaled_dot_product_attention(q, k, v)
        attention_mv = rearrange(
            attention[..., : self.hidden_dim * x.shape[-1]],
            "... (c x) -> ...  c x", x=x.shape[-1]
         )
        attention_mv = rearrange(
            attention_mv,
            "... heads items hidden_dim x -> ... items (heads hidden_dim) x"
         )

        outputs_mv = self.output_projection(attention_mv)

        return outputs_mv

In [ ]:
class GeometricBilinearLayer(nn.Module):
    """
    Geometric bilinear layer.

    Args:
        in_mv_channels (int): number of channels in the input multivector
        hidden_mv_channels (int): number of channels in the hidden multivector

    Attributes:
        geom_linear_1 (EquiLinearLayer): layer for geometric product operand 1
        geom_linear_2 (EquiLinearLayer): layer for geometric product operand 2
        join_linear_1 (EquiLinearLayer): layer for join operand 1
        join_linear_2 (EquiLinearLayer): layer for join operand 2

    Methods:
        forward(x, ref): computes the forward pass of the geometric bilinear layer
    """

    def __init__(self, in_mv_channels, hidden_mv_channels, blade, blade_len):
        super(GeometricBilinearLayer,self).__init__()

        self.geom_linear_1 = EquiLinearLayer(
            in_mv_channels = in_mv_channels,
            out_mv_channels = hidden_mv_channels // 2,
            blade = blade,
            blade_len = blade_len,
         )

        self.geom_linear_2 = EquiLinearLayer(
            in_mv_channels = in_mv_channels,
            out_mv_channels = hidden_mv_channels // 2,
            blade = blade,
            blade_len = blade_len,
         )

        self.join_linear_1 = EquiLinearLayer(
            in_mv_channels = in_mv_channels,
            out_mv_channels = hidden_mv_channels // 2,
            blade = blade,
            blade_len = blade_len,
         )

        self.join_linear_2 = EquiLinearLayer(
            in_mv_channels = in_mv_channels,
            out_mv_channels = hidden_mv_channels // 2,
            blade = blade,
            blade_len = blade_len,
         )

    def forward(self, x, ref):
        geom_linear_1 = self.geom_linear_1(x)
        geom_linear_2  = self.geom_linear_2(x)
        geom_product = geometric_product(
            geom_linear_1, geom_linear_2
        ) * 10e-6

        join_linear_1 = self.join_linear_1(x)
        join_linear_2 = self.join_linear_2(x)

        print("x", x.shape)
        print("geom_linear_1: ", geom_linear_1.shape)
        print("geom_linear_2: ", geom_linear_2.shape)
        print("geom_product: ", geom_product.shape)
        print("join_linear_1: ", join_linear_1.shape)
        print("join_linear_2: ", join_linear_2.shape)

        equi_join = join(join_linear_1, join_linear_2, ref)
        outputs_mv = torch.cat((geom_product, equi_join), dim=-2)

        return outputs_mv

In [ ]:
class GatedGELU(nn.Module):
    """
    Gated GeLU activation

    Attributes:
        gelu (nn.GELU): GELU activation function

    Methods:
        forward(x): computes the forward pass of the Gated GeLU layer
    """

    def __init__(self):
        super(GatedGELU,self).__init__()
        self.gelu = nn.GELU()
        self.scalar_gelu = nn.GELU()

    def forward(self,x):

        gates = x[..., [0]]
        weights = self.gelu(gates)
        outputs = weights * x

        return outputs

### Network

In [ ]:
class GATrNet(nn.Module):
    """
    GATr neural network.

    Args:
        in_channels (int): Number of input channels in the data
        blade (torch.Tensor): Blade tensor for EquilinearLayer operations
        blade_len (int): Length of the blade tensor
        hidden_dim (int): Hidden dimension of the EquilinearLayer

    Attributes:
        enter_equilinear (EquiLinearLayer): EquilinearLayer for initial projection
        attention_equinorm (EquilinearNormLayer): EquilinearNormLayer for attention block normalization
        geometric_attention (GeometricAttentionLayer): GeometricAttentionLayer for attention block
        bilinear_equinorm (EquilinearNormLayer): EquilinearNormLayer for bilinear block normalization
        geometric_bilinear (GeometricBilinearLayer): GeometricBilinearLayer for bilinear block
        gated_gelu (GatedGELU): Gated GELU activation function
        gelu_equilinear (EquiLinearLayer): EquilinearLayer for final GELU operation
        final_equilinear (EquiLinearLayer): Final EquilinearLayer for output projection
        vectorizer (nn.Linear): Linear layer for vectorization

    Methods:
        forward(x): computes the forward pass of the GATrNet
    """

    def __init__(self, in_channels, blade, blade_len, hidden_dim, attention_hidden_dim, n_heads, n_classes, crop_size):
        super(GATrNet, self).__init__()

        # Entering Equilinear Layer
        self.enter_equilinear = EquiLinearLayer(
            in_mv_channels = in_channels,
            out_mv_channels = hidden_dim,
            blade = blade,
            blade_len = blade_len
         )

        # Attention block
        self.attention_equinorm = EquilinearNormLayer()

        self.geometric_attention = GeometricAttentionLayer(
            mv_channels = hidden_dim,
            hidden_dim = attention_hidden_dim, # controllare che questa si possa modificare!!!!
            out_channels = hidden_dim, # controllare che questa si possa modificare!!!!
            blade = blade,
            blade_len = blade_len,
            n_heads = n_heads,
         )

        # Bilinear block
        self.bilinear_equinorm = EquilinearNormLayer()

        self.geometric_bilinear = GeometricBilinearLayer(
            in_mv_channels = hidden_dim,
            hidden_mv_channels = hidden_dim,
            blade = blade,
            blade_len = blade_len
         )

        self.gated_gelu = GatedGELU()

        self.gelu_equilinear = EquiLinearLayer(
            in_mv_channels = hidden_dim,
            out_mv_channels = hidden_dim,
            blade = blade,
            blade_len = blade_len
         )

        # Final Equilinear Layer
        self.final_equilinear = EquiLinearLayer(
            in_mv_channels = hidden_dim,
            out_mv_channels = n_classes,
            blade = blade,
            blade_len = blade_len
         )

        # Vectorizer
        self.vectorizer = nn.Linear(
            crop_size,
            n_classes,
         )

    def forward(self, x):
        # Entering Equilinear Layer
        projected_in_mv = self.enter_equilinear(x)

        # Attention block
        normalized_in_mv = self.attention_equinorm(
            projected_in_mv
         )

        attended_mv = self.geometric_attention(
            normalized_in_mv
         )

        # Residual connection 1
        residual_mv_1 = attended_mv + projected_in_mv

        # Bilinear block
        reference = torch.mean(x, dim = (1,2), keepdim = True)

        normalized_residual_mv = self.bilinear_equinorm(
            residual_mv_1
         )

        print("normalized residual", normalized_residual_mv.shape)
        print("Reference shape:", reference.shape)
        print("x shape: ", x.shape)

        bilinear_mv = self.geometric_bilinear(
            normalized_residual_mv,
            reference
         )

        gated_mv = self.gated_gelu(bilinear_mv)

        projected_bilinear_mv = self.gelu_equilinear(
            gated_mv,
         )

        # Residual connection 2
        residual_mv_2 = projected_bilinear_mv + residual_mv_1

        # Final Equilinear Layer
        output_mv = self.final_equilinear(residual_mv_2)

        extracted_scalars = output_mv[:,:,0,:][...,[0]].squeeze(-1)
        logits = self.vectorizer(extracted_scalars)

        return logits

## Modello Ufficiale

In [ ]:
!git clone https://github.com/Qualcomm-AI-research/geometric-algebra-transformer.git

Cloning into 'geometric-algebra-transformer'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 380 (delta 57), reused 47 (delta 47), pack-reused 286 (from 1)
Receiving objects: 100% (380/380), 383.13 KiB | 2.05 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [ ]:
%cd geometric-algebra-transformer

/content/geometric-algebra-transformer/geometric-algebra-transformer/geometric-algebra-transformer


In [ ]:
from gatr.nets import GATr
from gatr.layers.mlp import MLPConfig
from gatr.layers.attention.config import SelfAttentionConfig

In [ ]:
class GATrOfficial(nn.Module):
    def __init__(
            self,
            in_channels,
            hidden_dimension,
            out_channels,
            n_heads,
            crop_size,
            num_classes,
            in_s_channels = 0,
            num_blocks = 1,
            dropout_prob = 0.0,
            dropout_final = 0.0,
        ):

        super(GATrOfficial, self).__init__()

        att_config = SelfAttentionConfig()
        att_config.num_heads = n_heads
        att_config.pos_encoding = False

        if in_s_channels != 0:
          self.gatr = GATr(
            in_mv_channels = in_channels,
            out_mv_channels = out_channels,
            hidden_mv_channels = hidden_dimension,
            in_s_channels = in_s_channels,
            out_s_channels = 1,
            hidden_s_channels = 4,
            attention = att_config,
            mlp = MLPConfig(),
            num_blocks = num_blocks,
            dropout_prob = dropout_prob,
          )
        else:
          self.gatr = GATr(
            in_mv_channels = in_channels,
            out_mv_channels = out_channels,
            hidden_mv_channels = hidden_dimension,
            attention = att_config,
            mlp = MLPConfig(),
            num_blocks = num_blocks,
            dropout_prob = dropout_prob,
          )

        #self.flatten = EquiLinearLayer(blade, 9, out_channels, 1, dropout_p = 0.0)
        #self.classification = EquiLinearLayer(blade, 9, crop_size * crop_size, num_classes, dropout_p = 0.0)

        #self.classification = EquiLinearLayer(blade, 9, out_channels, num_classes) # <- quello sempre usato (15/07/2025)

        #self.dropout = nn.Dropout(p=0.1)
        #self.proj = nn.Linear(16, 1)

        self.classification = EquiLinearLayer(blade, 9, (crop_size * crop_size), 1, dropout_p = dropout_final)

        self.vectorizer = nn.Linear(
             16,
             1,
        )

        '''self.flatten = EquiLinearLayer(blade, 9, crop_size * crop_size * out_channels, num_classes, dropout_p = 0.0)
        self.vectorizer = nn.Linear(
            16,
            1,
         )'''

        '''self.vectorizer = nn.Linear(
            crop_size * crop_size,
            num_classes,
         )'''

    def forward(self, x, scalars = None):
        print(x.shape)
        out_mv, s = self.gatr(x, scalars)
        '''extracted_scalars = out_mv[:,:,0,:][...,[0]].squeeze(-1)
        out = self.vectorizer(extracted_scalars)'''

        '''cls_token_final = out_mv[:, 0, :, :]
        out = self.classification(cls_token_final)'''
        #out = out_mv[:, 0, :, :]

        '''out_mv = out_mv.view(out_mv.size(0), -1, out_mv.size(3))
        out_mv = self.flatten(out_mv)
        out = self.vectorizer(out_mv)
        out = out.squeeze(-1)'''

        '''out = self.flatten(out_mv)
        out = out.squeeze(-2)
        out = self.classification(out)

        out = self.proj(out)
        out = out.squeeze(-1)'''

        out = out_mv.permute(0, 2, 1, -1)
        out = self.classification(out)
        out = out.squeeze(-2)

        out = F.dropout(out, p = 0.3, training = self.training)
        out = self.vectorizer(out)
        out = out.squeeze(-1)

        return out

# ViGatr

## Model

### Embeddings

#### Patch to Multivector

In [ ]:
def extract_linear_patches(img, patch_length=4, stride=4, orientation='horizontal'):
    """
    Estrae patch lineari (orizzontali o verticali) da un'immagine B,C,H,W con stride specificato.
    Restituisce (B, N_patches, C, L) dove L è il numero di pixel nella patch per canale.

    Args:
        img (torch.Tensor): Tensor di input con shape (B, C, H, W)
        patch_length (int): Lunghezza della patch lineare
        stride (int): Stride per l'estrazione delle patch
        orientation (str): 'horizontal' per 1xL, 'vertical' per Lx1

    Returns:
        torch.Tensor: Tensor di output con shape (B, N_patches, C, L)
    """
    B, C, H, W = img.shape

    if orientation == 'horizontal':
        # Patch 1xK lungo l'asse W
        img_padded = F.pad(img, (2, 2, 0, 0))
        patches = img_padded.unfold(3, patch_length, stride)  # (B, C, H, num_patches_w, patch_length)
        patches = patches.permute(0, 3, 1, 2, 4)        # (B, num_patches_w, C, H, patch_length)
        patches = patches.contiguous().view(B, -1, C, patch_length)  # (B, N_patches, C, patch_length)

    elif orientation == 'vertical':
        # Patch Kx1 lungo l'asse H
        img_padded = F.pad(img, (0, 0, 2, 2))
        patches = img_padded.unfold(2, patch_length, stride)  # (B, C, num_patches_h, W, patch_length)
        patches = patches.permute(0, 2, 1, 3, 4)        # (B, num_patches_h, C, W, patch_length)
        patches = patches.contiguous().view(B, -1, C, patch_length)  # (B, N_patches, C, patch_length)

    else:
        raise ValueError("orientation must be either 'horizontal' or 'vertical'")

    return patches


In [ ]:
def extract_color_patches(img, patch_size=4, stride=4):
    """
    Estrae patch 4x4 da un'immagine B,C,H,W con stride specificato e le restituisce
    nel formato B, N_patches, C, 16 (dove 16 sono i valori dei pixel per patch per canale).

    Args:
        img (torch.Tensor): Tensor di input con shape (B, C, H, W)
        patch_size (int): Dimensione della patch (default 4)
        stride (int): Stride per l'estrazione delle patch

    Returns:
        torch.Tensor: Tensor di output con shape (B, N_patches, C, 16)
    """
    B, C, H, W = img.shape

    # Estrai patch su H e W separatamente per ottenere (B, C, num_patches_h, num_patches_w, patch_size, patch_size)
    patches = img.unfold(2, patch_size, stride).unfold(3, patch_size, stride)  # (B, C, H_out, W_out, patch_size, patch_size)

    # Combina le dimensioni spatiali delle patch
    patches = patches.contiguous().view(B, C, -1, patch_size * patch_size)  # (B, C, N_patches, 16)

    # Riordina in (B, N_patches, C, 16)
    patches = patches.permute(0, 2, 1, 3)  # (B, N_patches, C, 16)

    return patches

In [ ]:
def extract_multivector_by_channel(img, patch_size=4, stride=4):
    B, C, H, W = img.shape

    # Estrai patch su H e W separatamente per ottenere (B, C, num_patches_h, num_patches_w, patch_size, patch_size)
    patches = img.unfold(2, patch_size, stride).unfold(3, patch_size, stride)  # (B, C, H_out, W_out, patch_size, patch_size)

    B, C, H_out, W_out, patch_size, patch_size = patches.shape

    patches = patches.contiguous().view(B, C, H_out * W_out, patch_size * patch_size)
    patches = patches.permute(0, 2, 3, 1)  # (B, 16, N_patches, C)

    return patches

In [ ]:
import torch

def extract_3d_windows(x, size, window_size=(2, 2, 2), stride=(2, 2, 2)):
    # x: (B, C, H, W)
    B, C, H, W = x.shape
    x = x.unsqueeze(1)  # (B, 1, C, H, W)
    B, C, D, H, W = x.shape
    d_win, h_win, w_win = window_size

    patches = x.unfold(2, d_win, stride[0]) \
               .unfold(3, h_win, stride[1]) \
               .unfold(4, w_win, stride[2])  # (B, C, num_d, num_h, num_w, d_win, h_win, w_win)

    num_d, num_h, num_w = patches.shape[2:5]
    patch_size = d_win * h_win * w_win
    num_patches = num_d * num_h * num_w

    patches = patches.contiguous().view(B, C, num_patches, patch_size)
    return patches


def extract_3D_multivector(img, window_size, crop_size, stride=(2,2,2)):
    B, C, H, W = img.shape
    x = img.unsqueeze(1)  # diventa (B, 1, C, H, W)
    # Definisco il pooling 3D con kernel 2x2x2 e stride 2
    max_pool = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=stride)

    out_max = max_pool(x)  # (B, 1, C/2, H/2, W/2)
    out_min = -max_pool(-x)  # (B, 1, C/2, H/2, W/2)

    out_max = out_max.squeeze(1) # (B, C/2, H/2, W/2)
    out_min = out_min.squeeze(1) # (B, C/2, H/2, W/2)

    patches_max = extract_3d_windows(out_max, window_size)  # (B, n, 8)
    patches_min = extract_3d_windows(out_min, window_size)  # (B, n, 8)

    # Numero finestre
    n_windows = patches_max.size(1)
    patch_size = patches_max.size(2)

    # Crea array finale alternato
    patches = torch.stack([patches_max, patches_min], dim=-1)  # (B, 16, 16, 8, 2)
    patches = patches.view(B, crop_size * crop_size, 16, -1)  # (B, 16, 16, 16)
    patches = patches.to(device)

    patches = patches.permute(0, 1, 3, 2)
    return patches

In [ ]:
class ViGATrEmbeddingSimplePatchToMultivector(nn.Module):
    def __init__(self, stride, blade, blade_len, input_channels, output_dimension, type):
        super().__init__()
        self.stride = stride

        self.projection1 = EquiLinearLayer(
            blade=blade,
            blade_len=blade_len,
            in_mv_channels=input_channels,
            out_mv_channels=output_dimension
        )

        self.type = type

    def forward(self, x):
        if self.type == 'patch':
          res = extract_color_patches(x, 4, 4)
        elif self.type == 'horizontal':
          res = extract_linear_patches(x, 16, 16, 'horizontal')
        elif self.type == 'vertical':
          res = extract_linear_patches(x, 16, 16, 'vertical')

        #res = self.projection1(res)

        return res

In [ ]:
class ViGATrEmbeddingDoublePatchToMultivector(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x_1 = x[0]
        x_2 = x[1]
        res_1 = extract_color_patches(x_1, 4, 4)
        res_2 = extract_multivector_by_channel(x_2, 4, 4)
        res = torch.cat([res_1, res_2], dim=-2)

        return res

In [ ]:
class ViGATrEmbeddingTriplePatchToMultivector(nn.Module):
    def __init__(self, window_size, crop_size, dsm_embedding = 0):
        super().__init__()

        self.window_size = window_size
        self.crop_size = crop_size

        self.dsm_embedding = dsm_embedding
        if dsm_embedding:
          self.dsm_linear = nn.Linear(16, dsm_embedding)

    def forward(self, x):
        x_1 = x[0]
        x_2 = x[1]
        x_3 = x[2]
        res_1 = extract_color_patches(x_1, 4, 4)
        res_2 = extract_multivector_by_channel(x_2, 4, 4)
        _, _, _, C = res_2.shape
        chunks = torch.chunk(res_2, C // 16, dim=-1)  # ogni pezzo ha 16 canali

        res_3 = extract_3D_multivector(x_3, self.window_size, self.crop_size)
        #res = torch.cat([res_1, res_2, res_3], dim=-2)
        res = torch.cat([res_1, *chunks, res_3], dim=-2)

        scalars = None
        if self.dsm_embedding:
          patch_size = 4
          patches = x[3].unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
          B, C, num_patches_h, num_patches_w, ph, pw = patches.shape
          patches = patches.contiguous().view(B, C, num_patches_h * num_patches_w, ph * pw)

          B, C, N, P = patches.shape  # B=24, C=2, N=9, P=16

          # unisci i canali nell'ultima dimensione
          patches = patches.view(B, 1, N, C * P)  # [24, 1, 9, 32]

          patches = patches.squeeze(1)
          scalars = self.dsm_linear(patches)
          #scalars = patches

        return res, scalars

### Network

In [ ]:
class ViGatr(nn.Module):
    def __init__(
            self,
            in_channels,
            mv_in_channels,
            out_channels,
            blade,
            blade_len,
            hidden_dim,
            crop_size,
            n_classes,
            n_heads,
            positional_dim = 0,
            blocks = 1,
            dropout_gatr = 0,
            dropout_final = 0.0,
            stride = 3,
            type_embedding = "patch",
            window_size = 16,
            dsm_embedding = 0,
        ):

        super(ViGatr, self).__init__()

        self.positional_dim = positional_dim
        self.crop_size = crop_size
        self.dsm_embedding = dsm_embedding
        '''self.preprocessor = ViGATrEmbeddingSimplePatchToMultivector(
            stride = stride,
            blade = blade,
            blade_len = blade_len,
            input_channels = in_channels,
            output_dimension = mv_in_channels,
            type = type_embedding,
        )'''

        #self.preprocessor = ViGATrEmbeddingDoublePatchToMultivector()
        self.preprocessor = ViGATrEmbeddingTriplePatchToMultivector(window_size, crop_size, dsm_embedding)
        self.mv_channels = mv_in_channels
        self.gatr = GATrOfficial(
          in_channels = mv_in_channels,
          hidden_dimension = hidden_dim,
          out_channels = out_channels,
          n_heads = n_heads,
          crop_size = crop_size,
          num_classes = n_classes,
          in_s_channels = dsm_embedding + positional_dim,
          num_blocks = blocks,
          dropout_prob = dropout_gatr,
          dropout_final = dropout_final,
        )

        if positional_dim:
          #self.position_encoding = self.get_sinusoid_encoding(crop_size, crop_size, positional_dim).to(device)
          self.position_encoding = nn.Parameter(torch.randn(1, (crop_size * crop_size), positional_dim)) # <- levato il +1 15/07/2025

        self._initialize_weights()

        #self.pos_embed = nn.Parameter(torch.randn(1, (crop_size * crop_size) + 1, mv_in_channels, 16))
        #self.cls_token = nn.Parameter(torch.randn(1, 1, mv_in_channels, 16))  # token [CLS]


    def get_sinusoid_encoding(self, height, width, d_model):
        i = torch.arange(height).unsqueeze(1)  # (height, 1)
        j = torch.arange(width).unsqueeze(0)   # (1, width)

        # Calcola la codifica sinusoidale
        position_encoding = torch.zeros(height, width, d_model)
        for k in range(d_model // 2):
            freq = 1 / (10000 ** (2 * k / d_model))
            position_encoding[:, :, 2*k] = torch.sin(i * freq) + torch.sin(j * freq)
            position_encoding[:, :, 2*k+1] = torch.cos(i * freq) + torch.cos(j * freq)

        return position_encoding


    def _initialize_weights(self):
        """
        Funzione per inizializzare i pesi dei moduli del modello.
        Può essere estesa con altre strategie di inizializzazione.
        """
        for m in self.modules():
            if isinstance(m, nn.Linear):
                # Inizializzazione di Xavier (adatta per attivazioni ReLU)
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Conv2d):
                # Inizializzazione di He per i layer convoluzionali
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                # Inizializzazione della normalizzazione del layer
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x, scalars = self.preprocessor(x)

        #x = self.norm(x)
        #x = self.prepro_dropout(x)
        if self.positional_dim:
          #cls_tokens = self.cls_token.expand(x.size(0), -1, -1, -1).to(device)
          #x = torch.cat((cls_tokens, x), dim=1)  #<- fino al 15/07/2025
          #x = self.prepro_dropout(x)

          #positional_emb = self.get_sinusoid_encoding(self.crop_size, self.crop_size, self.positional_dim)
          #positional_emb = positional_emb.view(-1, self.dsm_embedding)            # (9, 128)
          #positional_emb = positional_emb.unsqueeze(0)  # (1, 1, 9, 128)
          #positional_emb = positional_emb.repeat(x.shape[0], 1, 1)       # (B, 1, 9, 128)
          #positional_emb = positional_emb.to(device)

          positional_emb = self.position_encoding.reshape(x.shape[1], self.positional_dim)
          positional_emb = positional_emb.unsqueeze(0).repeat(x.shape[0], 1, 1)
          if scalars != None:
            positional_emb = torch.cat((positional_emb, scalars), dim=-1)

          y = self.gatr(x, scalars = positional_emb)
        else:
          y = self.gatr(x, scalars)

        return y

## Pytorch Ligthtning

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_confusion_matrix(conf_matrix, class_names=None, normalize=False, title="Confusion Matrix"):
    cm = conf_matrix.cpu().numpy()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d", cmap='Blues',
                xticklabels=class_names if class_names else range(cm.shape[1]),
                yticklabels=class_names if class_names else range(cm.shape[0]))

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(title)
    plt.show()


In [ ]:
def cosine_similarity_classification_loss(output, label_multivectors, target):
    diff = output.unsqueeze(1) - label_multivectors.unsqueeze(0)  # shape: (B, num_classes, 16)

    # Calcola la norma (es. L2) lungo l'ultima dimensione
    dists = torch.norm(diff, dim=-1)  # shape: (B, num_classes)
    dists = dists.squeeze(-2)
    # Per usare cross_entropy, servono *logits*, non distanze ⇒ inverti il segno
    logits = -dists

    # Calcola la loss (usando y come indice della classe corretta)
    loss = F.cross_entropy(logits, target)

    return loss

In [ ]:
import torch
import numpy as np

def mixup_windows(batch_windows, labels, alpha=0.4):
    """
    batch_windows: lista di 3 tensori [B, C, H, W] -> [x1, x2, x3]
    labels: tensor [B] interi
    alpha: coeff beta
    """
    x1, x2, x3 = batch_windows
    B = x1.size(0)

    # Scegli λ dal beta
    lam = np.random.beta(alpha, alpha)

    # Indici random per mixare
    index = torch.randperm(B).to(x1.device)

    # Mix separato per ogni finestra
    mixed_x1 = lam * x1 + (1 - lam) * x1[index]
    mixed_x2 = lam * x2 + (1 - lam) * x2[index]
    mixed_x3 = lam * x3 + (1 - lam) * x3[index]

    # Label soft
    num_classes = 15
    y_a = torch.nn.functional.one_hot(labels, num_classes=num_classes).float()
    y_b = torch.nn.functional.one_hot(labels[index], num_classes=num_classes).float()
    mixed_labels = lam * y_a + (1 - lam) * y_b

    return [mixed_x1, mixed_x2, mixed_x3], mixed_labels


In [ ]:
import torch
import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    def __init__(self, weight=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Cross entropy base
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none')
        # Calcola p_t = probabilità assegnata alla classe corretta
        p_t = torch.exp(-ce_loss)
        # Applica il fattore di focalizzazione
        focal_loss = ((1 - p_t) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [ ]:
from collections import namedtuple
from torchmetrics import Precision, Recall
from torchvision.transforms.functional import rgb_to_grayscale
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from torchmetrics.classification import MulticlassAccuracy
from timm.data.mixup import Mixup
from timm.loss import SoftTargetCrossEntropy

class ViTGATr_LIGHT(pl.LightningModule):
    def __init__(
          self,
          model,
          n_classes,
          multivector_labels,
          learning_rate,
        ):

        super(ViTGATr_LIGHT, self).__init__()

        self.automatic_optimization = False

        self.accuracy = torchmetrics.Accuracy(task="multiclass", top_k=1, num_classes=n_classes)
        self.train_accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)

        self.accuracy_overall = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='micro')  # Overall accuracy
        self.accuracy_avg = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='macro')      # Per-class average accuracy
        self.kappa = torchmetrics.CohenKappa(task="multiclass", num_classes=n_classes)

        self.train_accuracy_overall = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='micro')  # Overall accuracy
        self.train_accuracy_avg = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='macro')      # Per-class average accuracy
        self.train_kappa = torchmetrics.CohenKappa(task="multiclass", num_classes=n_classes)

        self.test_accuracy_overall = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='micro')  # Overall accuracy
        self.test_accuracy_avg = torchmetrics.Accuracy(task="multiclass", num_classes=n_classes, average='macro')      # Per-class average accuracy
        self.test_kappa = torchmetrics.CohenKappa(task="multiclass", num_classes=n_classes)
        self.test_per_class_accuracy = MulticlassAccuracy(num_classes=n_classes, average=None)
        self.test_per_class_accuracies = None

        self.conf_matrix = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=n_classes)
        #self.confusion = torchmetrics.MulticlassConfusionMatrix(num_classes=n_classes)

        self.precision = Precision(task="multiclass", num_classes=n_classes, average="macro")
        self.recall = Recall(task="multiclass", num_classes=n_classes, average="macro")

        self.train_precision = Precision(task="multiclass", num_classes=n_classes, average="macro")
        self.train_recall = Recall(task="multiclass", num_classes=n_classes, average="macro")

        self.test_results = []

        self.class_names = data_class_names
        self.multivector_labels = multivector_labels

        self.learning_rate = learning_rate

        labels_list = [label for _, _, _, label in hsi_train_dataset.data_windows]
        unique_classes = np.unique(labels_list) # Get unique class labels
        class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=labels_list) # Pass unique_classes as a numpy array
        class_weights = torch.tensor(class_weights, dtype=torch.float32)
        class_weights = torch.clamp(class_weights, min=0.5, max=5.0)


        print("Class weights:", class_weights)
        print("Weight for trees (classe 0):", class_weights[0])

        self.loss_function = nn.CrossEntropyLoss(weight=class_weights.to(device), label_smoothing=0.1)
        #self.loss_function = cosine_similarity_classification_loss

        self.vit = model

        self.output = namedtuple('Output', ['logits'])

        '''self.mixup_fn = Mixup(
            mixup_alpha=0.2,
            cutmix_alpha=0.2,   # se vuoi solo mixup
            prob=0.8,           # probabilità di applicarlo
            label_smoothing=0.1,
            num_classes=15
        )'''

    def forward(self, x):
        logits = self.vit(x)

        '''
        logits = F.normalize(logits, dim=-1)
        labels = F.normalize(self.multivector_labels, dim=-1)

        diff = logits.unsqueeze(1) - self.multivector_labels.unsqueeze(0)  # shape: (B, num_classes, 16)
        #diff = logits - self.multivector_labels.unsqueeze(0)

        # Calcola la norma (es. L2) lungo l'ultima dimensione
        dists = torch.norm(diff, dim=-1)  # shape: (B, num_classes)
        dists = dists.squeeze(-2)
        # Per usare cross_entropy, servono *logits*, non distanze ⇒ inverti il segno
        temperature = 0.05
        logits = -dists / temperature

        #dot = torch.sum(faster_inner_product(logits, self.multivector_labels.unsqueeze(0)), dim=-1)  # shape: [32, 16]
        #temperature = 0.05
        #dot = dot/temperature
        '''

        return self.output(logits=logits)


    def training_step(self, batch, batch_idx):
        images, labels = batch

        logits = self(images).logits

        loss = self.loss_function(logits, labels)
        #loss = self.loss_function(logits, self.multivector_labels, labels)

        unique, counts = torch.unique(labels, return_counts=True)

        self.manual_backward(loss, retain_graph=True)

        optimizer = self.optimizers()  # Ottieni l'ottimizzatore
        optimizer.step()  # Esegui il passo di ottimizzazione
        optimizer.zero_grad()  # Azzera i gradienti

        '''diff = logits.unsqueeze(1) - self.multivector_labels.unsqueeze(0)  # shape: (B, num_classes, 16)

        # Calcola la norma (es. L2) lungo l'ultima dimensione
        dists = torch.norm(diff, dim=-1)  # shape: (B, num_classes)
        dists = dists.squeeze(-2)
        # Per usare cross_entropy, servono *logits*, non distanze ⇒ inverti il segno
        logits = -dists'''

        # Aggiorna l'accuracy con i nuovi dati
        self.train_accuracy.update(logits, labels)
        self.train_precision.update(logits, labels)
        self.train_recall.update(logits, labels)
        self.train_accuracy_overall.update(logits, labels)
        self.train_accuracy_avg.update(logits, labels)
        self.train_kappa.update(logits, labels)

        self.log("loss", loss, on_step=False, on_epoch=True, prog_bar=True, batch_size=images[0].shape[0])

        return loss


    def on_train_epoch_end(self):
        # Calcola la loss media dell'epoca
        avg_train_loss = self.trainer.callback_metrics["loss"].item()
        train_acc = self.train_accuracy.compute()
        train_precision = self.train_precision.compute()
        train_recall = self.train_recall.compute()
        train_accuracy_overall = self.train_accuracy_overall.compute()
        train_accuracy_avg = self.train_accuracy_avg.compute()
        train_kappa = self.train_kappa.compute()

        self.log("accuracy_overall", train_accuracy_overall, prog_bar=True)
        self.log("accuracy_avg", train_accuracy_avg)
        self.log("kappa", train_kappa)


        # Stampa i risultati alla fine dell'epoca
        print(
            f"[Train] Epoch: {self.current_epoch}  Loss: {avg_train_loss:.4f}  "
            f"Accuracy Overall: {train_accuracy_overall:.4f}  "
            f"Accuracy Avg: {train_accuracy_avg:.4f}  "
            f"Kappa: {train_kappa:.4f}"
        )

        # Reset delle metriche per la prossima epoca
        self.train_accuracy.reset()
        self.train_precision.reset()
        self.train_recall.reset()
        self.train_accuracy_overall.reset()
        self.train_accuracy_avg.reset()
        self.train_kappa.reset()



    def validation_step(self, batch):
        images, labels = batch

        logits = self(images).logits
        sample_loss = self.loss_function(logits, labels)

        #sample_loss = self.loss_function(logits, self.multivector_labels, labels).clone().detach()

        '''diff = logits.unsqueeze(1) - self.multivector_labels.unsqueeze(0)  # shape: (B, num_classes, 16)

        # Calcola la norma (es. L2) lungo l'ultima dimensione
        dists = torch.norm(diff, dim=-1)  # shape: (B, num_classes)
        dists = dists.squeeze(-2)
        # Per usare cross_entropy, servono *logits*, non distanze ⇒ inverti il segno
        logits = -dists'''

        # Update all metrics in the collection
        self.accuracy.update(logits, labels)
        self.conf_matrix.update(logits, labels)
        self.precision.update(logits, labels)
        self.recall.update(logits, labels)
        self.accuracy_overall.update(logits, labels)
        self.accuracy_avg.update(logits, labels)
        self.kappa.update(logits, labels)
        #self.confusion.update(logits, labels)

        # Log loss per batch
        self.log("val_loss", sample_loss, on_epoch=True, prog_bar=True, batch_size=images[0].shape[0])

        return {"loss": sample_loss}


    def on_validation_epoch_end(self):
        # Compute and log all metrics at the end of the epoch
        computed_accuracy = self.accuracy.compute()

        avg_val_loss = self.trainer.callback_metrics["val_loss"].item()

        computed_precision = self.precision.compute()
        computed_recall = self.recall.compute()
        computed_accuracy_overall = self.accuracy_overall.compute()
        computed_accuracy_avg = self.accuracy_avg.compute()
        computed_kappa = self.kappa.compute()

        self.log("val_accuracy_overall", computed_accuracy_overall, prog_bar=True)
        self.log("val_accuracy_avg", computed_accuracy_avg)
        self.log("val_kappa", computed_kappa)


        print(
            f"[Validation] Epoch: {self.current_epoch}  "
            f"Loss: {avg_val_loss:.4f}  "
            f"Accuracy Overall: {computed_accuracy_overall:.4f}  "
            f"Accuracy Avg: {computed_accuracy_avg:.4f}  "
            f"Kappa: {computed_kappa:.4f}"
        )

        conf_matrix = self.conf_matrix.compute()
        #plot_confusion_matrix(conf_matrix, class_names=self.class_names)
        #print(f"Matrice di Confusione:\n{conf_matrix}")

        # Reset metrics for the next epoch
        self.accuracy.reset()
        self.conf_matrix.reset()
        self.precision.reset()
        self.recall.reset()
        self.accuracy_overall.reset()
        self.accuracy_avg.reset()
        self.kappa.reset()


    def test_step(self, batch):
        images, labels = batch

        logits = self(images).logits

        '''diff = logits.unsqueeze(1) - self.multivector_labels.unsqueeze(0)  # shape: (B, num_classes, 16)

        # Calcola la norma (es. L2) lungo l'ultima dimensione
        dists = torch.norm(diff, dim=-1)  # shape: (B, num_classes)
        dists = dists.squeeze(-2)
        # Per usare cross_entropy, servono *logits*, non distanze ⇒ inverti il segno
        logits = -dists'''

        self.test_accuracy_overall.update(logits, labels)
        self.test_accuracy_avg.update(logits, labels)
        self.test_kappa.update(logits, labels)
        self.test_per_class_accuracy.update(logits, labels)

        return {"test_loss": self.loss_function(logits, labels)}


    def on_test_epoch_end(self):
        test_accuracy_overall = self.test_accuracy_overall.compute()
        test_accuracy_avg = self.test_accuracy_avg.compute()
        test_kappa = self.test_kappa.compute()
        per_class_acc = self.test_per_class_accuracy.compute()

        self.test_per_class_accuracies = per_class_acc.cpu().numpy()

        # Print a summary of top-k accuracy metrics and MRR
        print(
            f"[Test] accurracy {test_accuracy_overall:.4f}; "
            f"accuracy avg {test_accuracy_avg:.4f}; "
            f"kappa {test_kappa:.4f}"
        )

        self.test_results = [test_accuracy_overall.item(), test_accuracy_avg.item(), test_kappa.item()]

        print(f"[Test] Accuracy per classe:")
        for i, acc in enumerate(per_class_acc):
            print(f"  Classe {i} ({self.class_names[i]}): {acc:.4f}")

        # Reset metrics for the next epoch
        self.test_accuracy_overall.reset()
        self.test_accuracy_avg.reset()
        self.test_kappa.reset()
        self.test_per_class_accuracy.reset()


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        '''optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.trainer.max_epochs),
            'interval': 'epoch',
            'frequency': 1
        }
        return [optimizer], [scheduler]'''

In [ ]:
blade = blade_operator().to(device)

# Triple Gatr

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, stride, direction='square', patch_length=16, input_channels=0, output_channels=0):
        super().__init__()
        self.stride = stride
        self.direction = direction
        self.patch_length = patch_length

        self.input_channels = input_channels
        self.output_channels = output_channels

        if input_channels != 0:
            self.linear = EquiLinearLayer(
                blade=blade,
                blade_len=9,
                in_mv_channels=input_channels,
                out_mv_channels=output_channels
        )

    def forward(self, x):
        if self.direction == 'square':
            x = extract_color_patches(x, patch_size=4, stride=self.stride)
        elif self.direction in ['horizontal', 'vertical']:
            x = extract_linear_patches(x, patch_length=self.patch_length, stride=self.patch_length, orientation=self.direction)
        else:
            raise ValueError(f"Unknown patch embedding direction: {self.direction}")
        return x


class TripleViGatrLinear(nn.Module):
    def __init__(
        self,
        input_channels,
        mv_in_channels,
        out_channels,
        hidden_dim,
        crop_size,
        n_classes,
        n_heads,
        positional_dim=0,
        blocks=1,
        dropout_gatr=0,
        stride=4,
        device='cpu',  # aggiunto parametro per device
    ):
        super().__init__()
        self.device = device
        self.positional_dim = positional_dim

        # Preprocessors
        self.preprocessors = nn.ModuleList([
            PatchEmbedding(stride, direction, input_channels=input_channels, output_channels=mv_in_channels) for direction in ['square', 'horizontal', 'vertical']
        ])

        # GATr modules
        self.gatrs = nn.ModuleList([
            GATrOfficial(
                in_channels=mv_in_channels,
                hidden_dimension=hidden_dim,
                out_channels=out_channels,
                n_heads=n_heads,
                crop_size=crop_size,
                num_classes=n_classes,
                in_s_channels=positional_dim,
                num_blocks=blocks,
                dropout_prob=dropout_gatr,
            ) for _ in range(3)
        ])

        self.encodings = []
        if positional_dim:
            self.encodings.append(nn.Parameter(torch.randn(1, (crop_size * crop_size) + 1, positional_dim)))
            self.encodings.append(nn.Parameter(torch.randn(1, 57, positional_dim)))
            self.encodings.append(nn.Parameter(torch.randn(1, 57, positional_dim)))
            #self.position_encoding = self.get_sinusoid_encoding(crop_size, crop_size, positional_dim).to(device)

        '''self.projection = EquiLinearLayer(
            blade=blade,
            blade_len=9,
            in_mv_channels=3*n_classes,
            out_mv_channels=n_classes,
        )'''

        self.cls_token = nn.Parameter(torch.randn(1, 1, mv_in_channels, 16))
        self.projection = nn.Linear(3*n_classes, n_classes)


    def get_sinusoid_encoding(self, height, width, d_model):
        i = torch.arange(height).unsqueeze(1)  # (height, 1)
        j = torch.arange(width).unsqueeze(0)   # (1, width)

        # Calcola la codifica sinusoidale
        position_encoding = torch.zeros(height, width, d_model)
        for k in range(d_model // 2):
            freq = 1 / (10000 ** (2 * k / d_model))
            position_encoding[:, :, 2*k] = torch.sin(i * freq) + torch.sin(j * freq)
            position_encoding[:, :, 2*k+1] = torch.cos(i * freq) + torch.cos(j * freq)

        return position_encoding


    def forward(self, x):
        outputs = []
        for i in range(3):
            xi = self.preprocessors[i](x)
            if self.positional_dim:
                cls_tokens = self.cls_token.expand(xi.size(0), -1, -1, -1).to(device)
                xi = torch.cat((cls_tokens, xi), dim=1)

                pe = self.encodings[i].reshape(xi.shape[1], self.positional_dim)
                pe = pe.unsqueeze(0).repeat(xi.shape[0], 1, 1).to(self.device)

                yi = self.gatrs[i](xi, scalars=pe)
            else:
                yi = self.gatrs[i](xi)

            yi = F.dropout(yi, p=0.7, training=self.training)
            outputs.append(yi)

        y = torch.cat(outputs, dim=-1)
        y = self.projection(y)

        return y

# Loop

In [ ]:
name = f"test-{pre_name}-{date_str}-N{loop_counter}"
loop_counter += 1

In [ ]:
!mkdir /content/{name}

In [ ]:
parameters = dict({
    # Pre processing
    "batch_size": 8,
    "window_size": 28,
    "pca_channels": 40,

    # Vit
    "in_channels": 1,
    "out_channels": 16,
    "hidden_dim": 16,
    "n_heads": 8,
    "positional_dim": 120,
    "mv_in_channels": 40,
    "blocks": 1,
    "dropout_gatr": 0.4,
    "stride": 4,
    "type_embedding": "patch",

    # Training
    "lr": 1e-2
})

In [ ]:
# Converto il dizionario in stringa formattata
text = ""
for k, v in parameters.items():
    text += f"{k}: {v}\n"

# Salvo su file
with open("/content/" + name + "/parameters.txt", "w") as f:
    f.write(text)


## Dataset

In [ ]:
window_size = parameters["window_size"]
batch_size = parameters["batch_size"]
pca_channels = parameters["pca_channels"]
n_classes = n_classes

In [ ]:
# prompt: Come posso ridurre il numero di canali dei miei dati da (145, 145, 200) a (145, 145, 80) usando PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_reshaped)

# Applica la PCA per ridurre i canali a 30
n_components = pca_channels
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(data_scaled)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced = data_pca.reshape(data.shape[0], data.shape[1], n_components)
data_reduced = data_reduced.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced.shape}")

In [ ]:
data_reduced = torch.from_numpy(data_reduced)
labels = torch.from_numpy(gt)

In [ ]:
tot_labels = 0
for row in labels:
  for column in row:
    if column != 0:
      tot_labels += 1
print("Numero totale di labels: ", tot_labels)
print("10% di tutti i labels: ", round(tot_labels / 100 * 10))

In [ ]:
if ten_per:
    # prompt: 1) Calcolare i labels diversi da 10
    # 2) Prendere il 10% dei labels diversi da 10 in modo randomico
    # 3) Ottenere di questi labels il corrispettivo pixel da data

    import numpy as np
    # 1) Calcolare i labels diversi da 0
    # Ottieni gli indici dei labels diversi da 0
    non_zero_labels_indices = np.where(labels.numpy() != 0)

    # Combina gli indici di riga e colonna in una lista di coordinate (riga, colonna)
    non_zero_labels_coords = list(zip(non_zero_labels_indices[0], non_zero_labels_indices[1]))

    print(f"Numero totale di labels diversi da 10: {len(non_zero_labels_coords)}")

    # 2) Prendere il 10% dei labels diversi da 10 in modo randomico
    import random

    # Calcola il 10% del numero totale di labels diversi da 10
    num_samples = int(len(non_zero_labels_coords) * 0.1)

    # Seleziona randomicamente gli indici
    random_non_zero_labels_coords = random.sample(non_zero_labels_coords, num_samples)

    mask = torch.zeros_like(labels, dtype=torch.bool)

    # Imposta a True i pixel selezionati casualmente
    for row, col in random_non_zero_labels_coords:
      mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero di True nella maschera:", mask.sum().item())

In [ ]:
import random

if not ten_per:
    # Supponiamo che labels sia un tensore PyTorch
    labels_np = labels.numpy()

    # Ottieni le classi uniche diverse da 0
    unique_classes = np.unique(labels_np)
    unique_classes = unique_classes[unique_classes != 0]

    # Crea una maschera vuota
    mask = torch.zeros_like(labels, dtype=torch.bool)

    for cls in unique_classes:
        # Trova gli indici (riga, colonna) dei pixel di questa classe
        cls_indices = np.where(labels_np == cls)
        cls_coords = list(zip(cls_indices[0], cls_indices[1]))

        # Se ci sono meno di 100 pixel in quella classe, prendili tutti
        n_samples = min(100, len(cls_coords))

        # Seleziona randomicamente fino a 100 pixel
        selected_coords = random.sample(cls_coords, n_samples)

        # Imposta a True nella maschera
        for row, col in selected_coords:
            mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero totale di True nella maschera:", mask.sum().item())


In [ ]:
labels_recap = np.zeros(n_classes)

final_labels = labels * mask
for row in final_labels:
  for column in row:
    if column != 0:
      labels_recap[column - 1] += 1

print(labels_recap)


In [ ]:
# prompt: Ora voglio creare un dataset composot di "immagini" e label:
# 1) Le immagini sono una finistra sull'immagine originale di dimensioni 16x16
# 2) Queste finestre sono ottenute per ognuno dei pixel che sono true in mask, e rispetto alla finestra questi devono trovarsi in posizione (8,8)
# 3) Se la finestra esce fuori dall'immagine impostare tutti i pixel e tutti canali fuori dall'immagine a 0
# 4) Alla finestra deve essere associato il label corrispondente (nel dataset si accede con x alla finestra e a y al label)

def create_dataset(image, mask, window_size=16):
    """
    Crea un dataset di finestre di immagini centrate sui pixel della maschera.

    Args:
        image (torch.Tensor): L'immagine iperspettrale ridotta (H, W, C).
        mask (torch.Tensor): La maschera booleana (H, W).
        window_size (int): La dimensione della finestra (window_size x window_size).

    Returns:
        list: Una lista di tuple (finestra, label).
    """
    dataset = []
    c, h, w = image.shape
    pad = window_size // 2  # Padding necessario per centrare il pixel

    # Applica un padding all'immagine per gestire i bordi
    # Il padding è necessario su altezza e larghezza, e su tutti i canali (si usa 0 per i canali)
    padded_image = torch.nn.functional.pad(image, (pad, pad, pad, pad), "constant", 0)
    print("Padded image shape: ", padded_image.shape)

    # Trova le coordinate (riga, colonna) dei pixel True nella maschera
    rows, cols = torch.where(mask)

    for r, c in zip(rows, cols):
        # Calcola le coordinate della finestra nell'immagine con padding
        # Il pixel di interesse (r, c) nell'immagine originale corrisponde a (r + pad, c + pad) nell'immagine con padding
        start_row = r + pad - pad
        end_row = r + pad + pad
        start_col = c + pad - pad
        end_col = c + pad + pad

        # Estrai la finestra dall'immagine con padding
        window = padded_image[:, start_row:end_row, start_col:end_col]

        # Ottieni il label dal tensore originale delle etichette
        label = labels[r, c].item() - 1

        # Aggiungi la finestra e il label al dataset
        dataset.append((window, label))

    return dataset

# Crea il dataset utilizzando l'immagine ridotta e la maschera creata in precedenza
dataset = create_dataset(data_reduced, mask, window_size=window_size)

print(f"Numero di esempi nel dataset: {len(dataset)}")

# Esempio di accesso al primo elemento del dataset
if len(dataset) > 0:
    first_window, first_label = dataset[0]
    print(f"Forma della prima finestra: {first_window.shape}")
    print(f"Label del primo esempio: {first_label}")


In [ ]:
# prompt: tramite dataset vorrei creare la classe dataset e dataloader

from torch.utils.data import Dataset, DataLoader

class HSI_Dataset(Dataset):
    """
    Dataset personalizzato per l'immagine iperspettrale,
    utilizzando le finestre generate dalla funzione create_dataset.
    """
    def __init__(self, data_windows):
        """
        Args:
            data_windows (list): Lista di tuple (finestra, label) generata da create_dataset.
        """
        self.data_windows = data_windows

    def __len__(self):
        return len(self.data_windows)

    def __getitem__(self, idx):
        window, label = self.data_windows[idx]
        # Converti la finestra in float32 per l'input del modello
        window = window.float()
        # Converti il label in LongTensor (per le funzioni di loss come CrossEntropyLoss)
        label = torch.tensor(label, dtype=torch.long)
        return window, label

# Crea un'istanza del dataset
hsi_train_dataset = HSI_Dataset(dataset)

print(f"Dimensione totale del dataset HSI: {len(hsi_train_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = True  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
train_dataloader = DataLoader(hsi_train_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(train_dataloader)}")

# Esempio di iterazione attraverso il DataLoader
# for batch_windows, batch_labels in dataloader:
#     print(f"Forma del batch di finestre: {batch_windows.shape}")
#     print(f"Forma del batch di labels: {batch_labels.shape}")
#     break # Esci dopo il primo batch per dimostrazione



In [ ]:
# prompt: ora però devo creare il test set con gli altri pixel che non ho usato

# Creazione del test set con i pixel rimanenti

# 1) Identificare i pixel che NON sono stati usati nel training set (maschera è False)
# Creiamo una maschera per il test set, che è l'inverso della maschera di training
test_mask = ~mask

print("Forma della maschera per il test set:", test_mask.shape)
print("Numero di True nella maschera del test set:", test_mask.sum().item())

# 2) Filtrare i pixel del test_mask per includere solo quelli con label > 0
# Vogliamo che il test set contenga solo pixel con classi effettive, non lo sfondo (label 0)
# Quindi, creiamo una maschera combinata dove sia test_mask che (labels > 0) sono True
test_mask_filtered = test_mask & (labels > 0)

print("Forma della maschera per il test set filtrata (solo pixel con label > 0):", test_mask_filtered.shape)
print("Numero di True nella maschera filtrata:", test_mask_filtered.sum().item())


# 3) Utilizzare la funzione create_dataset per creare il test set
# Passiamo l'immagine ridotta e la nuova maschera filtrata
test_dataset_windows = create_dataset(data_reduced, test_mask_filtered, window_size=window_size)

n_val = int(len(test_dataset_windows) * 0.1)
indices = np.random.permutation(len(test_dataset_windows))

val_idx = indices[:n_val]
test_idx = indices[n_val:]

val_dataset = [test_dataset_windows[i] for i in val_idx]
test_dataset = [test_dataset_windows[i] for i in test_idx]

print(f"Numero di esempi nel test set: {len(test_dataset)}")

# 4) Creare un'istanza della classe HSI_Dataset per il test set
hsi_test_dataset = HSI_Dataset(test_dataset)
hsi_val_dataset = HSI_Dataset(val_dataset)

print(f"Dimensione totale del test set HSI: {len(hsi_test_dataset)}")

# 5) (Opzionale) Creare un DataLoader per il test set
# Solitamente non si mescolano i dati nel test set
test_batch_size = 32
test_shuffle = False

test_dataloader = DataLoader(hsi_test_dataset, batch_size=test_batch_size, shuffle=test_shuffle)
val_dataloader = DataLoader(hsi_val_dataset, batch_size=test_batch_size, shuffle=test_shuffle)

print(f"Numero di batch nel DataLoader per il test set: {len(test_dataloader)}")

# Ora hai il tuo test set pronto per essere utilizzato per valutare il modello.
# Puoi iterare su test_dataloader per ottenere batch di dati e label per il testing.

In [ ]:
!mkdir /content/{name}/dataset

In [ ]:
output_path = "/content/" + name + "/dataset/labels.txt"
with open(output_path, "w") as f:
  f.write(f"Distribuzione labels: {labels_recap}\n")

In [ ]:
torch.save(hsi_train_dataset.data_windows, "/content/"+ name +"/dataset/hsi_train_dataset.pt")
torch.save(hsi_val_dataset.data_windows, "/content/"+ name +"/dataset/hsi_val_dataset.pt")
torch.save(hsi_test_dataset.data_windows, "/content/"+ name +"/dataset/hsi_test_dataset.pt")

## Train

In [ ]:
vit = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
)

In [ ]:
log_dir = "/content/"+name+"/logs"
csv_logger = CSVLogger(save_dir=log_dir, name="")

In [ ]:
vitgatr_mnist = ViTGATr_LIGHT(
    model = vit,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.0, patience=80, verbose=False, mode="min")
'''checkpoint_callback = ModelCheckpoint(
    dirpath="/content/"+ name + "/checkpoints/",         # directory dove salvare
    filename="hci",  # nome file
    save_top_k=1,                   # salva solo il migliore (o tutti)
    monitor="loss",            # metrica da monitorare
    mode="min",                    # "min" o "max" a seconda della metrica
)'''
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci-{epoch:02d}-{accuracy:.3f}",
    monitor="accuracy_overall",
    mode="max",
    save_top_k=-1,   # salva tutti quelli che superano la soglia
)

checkpoint_loss = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci",
    monitor="val_loss",
    mode="min",
    save_top_k=1
)


In [ ]:
trainer_mnist = Trainer(logger=csv_logger, max_epochs=300, callbacks=[checkpoint_callback, checkpoint_loss, early_stop_callback])
trainer_mnist.fit(vitgatr_mnist, train_dataloader, val_dataloader)

## Test

In [ ]:
from fvcore.nn import parameter_count
params = parameter_count(vitgatr_mnist)

print(f"Params: {params[''] / 1e6:.2f} M")

In [ ]:
trainer_mnist.test(vitgatr_mnist, dataloaders=test_dataloader, verbose=False)
results_last = vitgatr_mnist.test_results
print(results_last)

accuracies_last = vitgatr_mnist.test_per_class_accuracies

In [ ]:
vit_l = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
)

In [ ]:
vitgatr_mnist_l = ViTGATr_LIGHT.load_from_checkpoint(
    "/content/" + name + "/checkpoints/hci.ckpt",
    model = vit_l,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

In [ ]:
trainer_mnist.test(vitgatr_mnist_l, dataloaders=test_dataloader, verbose=False)
results_loaded = vitgatr_mnist_l.test_results
print(results_loaded)

accuracies_loaded = vitgatr_mnist_l.test_per_class_accuracies

In [ ]:
output_path = "/content/" + name + "/results.txt"
with open(output_path, "w") as f:
    f.write(f"Params: {params[''] / 1e6:.3f} M\n\n")
    f.write(f"Results last\n")
    f.write(f"Accuracy Overall: {results_last[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_last[1]:.4f}\n")
    f.write(f"Kappa: {results_last[2]:.4f}\n\n")
    f.write(f"Results loaded\n")
    f.write(f"Accuracy Overall: {results_loaded[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_loaded[1]:.4f}\n")
    f.write(f"Kappa: {results_loaded[2]:.4f}\n")

    for i, acc in enumerate(accuracies_loaded):
        f.write(f"Classe {i} ({vitgatr_mnist_l.class_names[i]}): {acc:.4f}\n")

    for i, acc in enumerate(accuracies_last):
        f.write(f"Classe {i} ({vitgatr_mnist.class_names[i]}): {acc:.4f}\n")

## Download

In [ ]:
!cp -r /content/{name} /content/drive/MyDrive/

# Loop doppi canali

In [ ]:
name = f"test-{pre_name}-{date_str}-N{loop_counter}"
loop_counter += 1

In [ ]:
!mkdir /content/{name}

In [ ]:
parameters = dict({
    # Pre processing
    "batch_size": 8,
    "window_size": 16,
    "pca_channels": 16,

    # Vit
    "in_channels": 1,
    "out_channels": 16,
    "hidden_dim": 16,
    "n_heads": 8,
    "positional_dim": 120,
    "mv_in_channels": 16 + 16,
    "blocks": 1,
    "dropout_gatr": 0.1,
    "stride": 4,
    "type_embedding": "patch",

    # Training
    "lr": 1e-3
})

In [ ]:
# Converto il dizionario in stringa formattata
text = ""
for k, v in parameters.items():
    text += f"{k}: {v}\n"

# Salvo su file
with open("/content/" + name + "/parameters.txt", "w") as f:
    f.write(text)


## Dataset

In [ ]:
window_size = parameters["window_size"]
batch_size = parameters["batch_size"]
pca_channels = parameters["pca_channels"]
n_classes = n_classes

In [ ]:
# prompt: Come posso ridurre il numero di canali dei miei dati da (145, 145, 200) a (145, 145, 80) usando PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_reshaped)

# Applica la PCA per ridurre i canali a 30
n_components = pca_channels
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(data_scaled)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced = data_pca.reshape(data.shape[0], data.shape[1], n_components)
data_reduced = data_reduced.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced.shape}")

In [ ]:
# prompt: Come posso ridurre il numero di canali dei miei dati da (145, 145, 200) a (145, 145, 80) usando PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped_2 = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled_2 = scaler.fit_transform(data_reshaped_2)

# Applica la PCA per ridurre i canali a 30
n_components = 16
pca = PCA(n_components=n_components)
data_pca_2 = pca.fit_transform(data_scaled_2)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced_2 = data_pca_2.reshape(data.shape[0], data.shape[1], n_components)
data_reduced_2 = data_reduced_2.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced_2.shape}")

In [ ]:
data_reduced = torch.from_numpy(data_reduced)
data_reduced_2 = torch.from_numpy(data_reduced_2)
labels = torch.from_numpy(gt)

In [ ]:
tot_labels = 0
for row in labels:
  for column in row:
    if column != 0:
      tot_labels += 1
print("Numero totale di labels: ", tot_labels)
print("10% di tutti i labels: ", round(tot_labels / 100 * 10))

In [ ]:
import random
if ten_per and global_ten_per:
    # prompt: 1) Calcolare i labels diversi da 10
    # 2) Prendere il 10% dei labels diversi da 10 in modo randomico
    # 3) Ottenere di questi labels il corrispettivo pixel da data

    import numpy as np
    # 1) Calcolare i labels diversi da 0
    # Ottieni gli indici dei labels diversi da 0
    non_zero_labels_indices = np.where(labels.numpy() != 0)

    # Combina gli indici di riga e colonna in una lista di coordinate (riga, colonna)
    non_zero_labels_coords = list(zip(non_zero_labels_indices[0], non_zero_labels_indices[1]))

    print(f"Numero totale di labels diversi da 10: {len(non_zero_labels_coords)}")

    # 2) Prendere il 10% dei labels diversi da 10 in modo randomico
    import random

    # Calcola il 10% del numero totale di labels diversi da 10
    num_samples = int(len(non_zero_labels_coords) * 0.1)

    # Seleziona randomicamente gli indici
    random_non_zero_labels_coords = random.sample(non_zero_labels_coords, num_samples)

    mask = torch.zeros_like(labels, dtype=torch.bool)

    # Imposta a True i pixel selezionati casualmente
    for row, col in random_non_zero_labels_coords:
      mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero di True nella maschera:", mask.sum().item())


if ten_per and not global_ten_per:
    labels_np = labels.numpy()
    unique_labels = np.unique(labels_np)

    mask = torch.zeros_like(labels, dtype=torch.bool)

    for label in unique_labels:
        if label == 0:  # oppure `if label == 10` se intendevi escludere la classe 10
            continue

        # Trova le coordinate dei pixel con questa classe
        indices = np.where(labels_np == label)
        coords = list(zip(indices[0], indices[1]))

        # Calcola quanti pixel prendere
        num_samples = max(1, int(len(coords) * 0.1))  # almeno 1 se pochi pixel

        if len(coords) > 0:
            sampled_coords = random.sample(coords, num_samples)
            for row, col in sampled_coords:
                mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero di True nella maschera:", mask.sum().item())

In [ ]:
import random

if not ten_per:
    # Supponiamo che labels sia un tensore PyTorch
    labels_np = labels.numpy()

    # Ottieni le classi uniche diverse da 0
    unique_classes = np.unique(labels_np)
    unique_classes = unique_classes[unique_classes != 0]

    # Crea una maschera vuota
    mask = torch.zeros_like(labels, dtype=torch.bool)

    for cls in unique_classes:
        # Trova gli indici (riga, colonna) dei pixel di questa classe
        cls_indices = np.where(labels_np == cls)
        cls_coords = list(zip(cls_indices[0], cls_indices[1]))

        # Se ci sono meno di 100 pixel in quella classe, prendili tutti
        n_samples = min(100, len(cls_coords))

        # Seleziona randomicamente fino a 100 pixel
        selected_coords = random.sample(cls_coords, n_samples)

        # Imposta a True nella maschera
        for row, col in selected_coords:
            mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero totale di True nella maschera:", mask.sum().item())


In [ ]:
labels_recap = np.zeros(n_classes)

final_labels = labels * mask
for row in final_labels:
  for column in row:
    if column != 0:
      labels_recap[column - 1] += 1

print(labels_recap)


In [ ]:
import torch
import random

class HSI_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_windows, augmented = False):
        """
        Args:
            data_windows (list): Lista di tuple (finestra, label) generata da create_dataset.
        """
        self.data_windows = data_windows
        self.augmented = augmented

    def __len__(self):
        return len(self.data_windows)

    def __getitem__(self, idx):

        window, window_2, label = self.data_windows[idx]
        # Converti la finestra in float32 per l'input del modello
        window = window.float()
        window_2 = window_2.float()

        if self.augmented:
            window, window_2 = self.apply_augmentation(window, window_2)

        # Converti il label in LongTensor (per le funzioni di loss come CrossEntropyLoss)
        label = torch.tensor(label, dtype=torch.long)
        return [window, window_2], label

    def apply_augmentation(self, x1, x2):
        # x1, x2: (C, H, W)
        if random.random() < 0.5:
            x1 = torch.flip(x1, dims=[1])  # Vertical flip
            x2 = torch.flip(x2, dims=[1])
        if random.random() < 0.5:
            x1 = torch.flip(x1, dims=[2])  # Horizontal flip
            x2 = torch.flip(x2, dims=[2])
        rot_k = random.choice([0, 1, 2, 3])  # 0°, 90°, 180°, 270°
        x1 = torch.rot90(x1, k=rot_k, dims=[1, 2])
        x2 = torch.rot90(x2, k=rot_k, dims=[1, 2])
        return x1, x2


In [ ]:
# prompt: Ora voglio creare un dataset composot di "immagini" e label:
# 1) Le immagini sono una finistra sull'immagine originale di dimensioni 16x16
# 2) Queste finestre sono ottenute per ognuno dei pixel che sono true in mask, e rispetto alla finestra questi devono trovarsi in posizione (8,8)
# 3) Se la finestra esce fuori dall'immagine impostare tutti i pixel e tutti canali fuori dall'immagine a 0
# 4) Alla finestra deve essere associato il label corrispondente (nel dataset si accede con x alla finestra e a y al label)

def create_dataset(image, image_2, mask, window_size=16):
    """
    Crea un dataset di finestre di immagini centrate sui pixel della maschera.

    Args:
        image (torch.Tensor): L'immagine iperspettrale ridotta (H, W, C).
        mask (torch.Tensor): La maschera booleana (H, W).
        window_size (int): La dimensione della finestra (window_size x window_size).

    Returns:
        list: Una lista di tuple (finestra, label).
    """
    dataset = []
    c, h, w = image.shape
    pad = window_size // 2  # Padding necessario per centrare il pixel

    # Applica un padding all'immagine per gestire i bordi
    # Il padding è necessario su altezza e larghezza, e su tutti i canali (si usa 0 per i canali)
    padded_image = torch.nn.functional.pad(image, (pad, pad, pad, pad), "replicate")
    padded_image_2 = torch.nn.functional.pad(image_2, (pad, pad, pad, pad), "replicate")
    print("Padded image shape: ", padded_image.shape)
    print("Padded image shape2: ", padded_image_2.shape)

    # Trova le coordinate (riga, colonna) dei pixel True nella maschera
    rows, cols = torch.where(mask)

    for r, c in zip(rows, cols):
        # Calcola le coordinate della finestra nell'immagine con padding
        # Il pixel di interesse (r, c) nell'immagine originale corrisponde a (r + pad, c + pad) nell'immagine con padding
        start_row = r + pad - pad
        end_row = r + pad + pad
        start_col = c + pad - pad
        end_col = c + pad + pad

        # Estrai la finestra dall'immagine con padding
        window = padded_image[:, start_row:end_row, start_col:end_col]
        window_2 = padded_image_2[:, start_row:end_row, start_col:end_col]

        # Ottieni il label dal tensore originale delle etichette
        label = labels[r, c].item() - 1

        # Aggiungi la finestra e il label al dataset
        dataset.append((window, window_2, label))

    return dataset

# Crea il dataset utilizzando l'immagine ridotta e la maschera creata in precedenza
dataset = create_dataset(data_reduced, data_reduced_2, mask, window_size=window_size)

print(f"Numero di esempi nel dataset: {len(dataset)}")

# Esempio di accesso al primo elemento del dataset
if len(dataset) > 0:
    first_window, second_window, first_label = dataset[0]
    print(f"Forma della prima finestra: {first_window.shape}")
    print(f"Label del primo esempio: {first_label}")


In [ ]:
if pre_name == 'ip':
    to_push = []
    for element in dataset:
        label = element[2]
        if label == 0 or label == 6 or label == 8:
          window1 = element[0]
          window2 = element[1]

          rot90_w1 = torch.rot90(window1, k=-1, dims=[1, 2])
          rot90_w2 = torch.rot90(window2, k=-1, dims=[1, 2])

          rotm90_w1 = torch.rot90(window1, k=1, dims=[1, 2])
          rotm90_w2 = torch.rot90(window2, k=1, dims=[1, 2])

          flipV_w1 = torch.flip(window1, dims=[1, 2])
          flipV_w2 = torch.flip(window2, dims=[1, 2])

          to_push.append((rot90_w1, rot90_w2, label))
          to_push.append((rotm90_w1, rotm90_w2, label))
          to_push.append((flipV_w1, flipV_w2, label))

    for element in to_push:
        dataset.append(element)

    labels_recap = np.zeros(n_classes)

    for element in dataset:
          labels_recap[element[2]] += 1

    print(labels_recap)

In [ ]:
# prompt: tramite dataset vorrei creare la classe dataset e dataloader

from torch.utils.data import Dataset, DataLoader

# Crea un'istanza del dataset
hsi_train_dataset = HSI_Dataset(dataset, augmented=True)

print(f"Dimensione totale del dataset HSI: {len(hsi_train_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = True  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
train_dataloader = DataLoader(hsi_train_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(train_dataloader)}")

# Esempio di iterazione attraverso il DataLoader
# for batch_windows, batch_labels in dataloader:
#     print(f"Forma del batch di finestre: {batch_windows.shape}")
#     print(f"Forma del batch di labels: {batch_labels.shape}")
#     break # Esci dopo il primo batch per dimostrazione



In [ ]:
# prompt: ora però devo creare il test set con gli altri pixel che non ho usato

# Creazione del test set con i pixel rimanenti

# 1) Identificare i pixel che NON sono stati usati nel training set (maschera è False)
# Creiamo una maschera per il test set, che è l'inverso della maschera di training
test_mask = ~mask

print("Forma della maschera per il test set:", test_mask.shape)
print("Numero di True nella maschera del test set:", test_mask.sum().item())

# 2) Filtrare i pixel del test_mask per includere solo quelli con label > 0
# Vogliamo che il test set contenga solo pixel con classi effettive, non lo sfondo (label 0)
# Quindi, creiamo una maschera combinata dove sia test_mask che (labels > 0) sono True
test_mask_filtered = test_mask & (labels > 0)

print("Forma della maschera per il test set filtrata (solo pixel con label > 0):", test_mask_filtered.shape)
print("Numero di True nella maschera filtrata:", test_mask_filtered.sum().item())


# 3) Utilizzare la funzione create_dataset per creare il test set
# Passiamo l'immagine ridotta e la nuova maschera filtrata
test_dataset_windows = create_dataset(data_reduced, data_reduced_2, test_mask_filtered, window_size=window_size)

n_val = int(len(test_dataset_windows) * 0.1)
indices = np.random.permutation(len(test_dataset_windows))

val_idx = indices[:n_val]
test_idx = indices[n_val:]

val_dataset = [test_dataset_windows[i] for i in val_idx]
test_dataset = [test_dataset_windows[i] for i in test_idx]

'''from collections import defaultdict

# Raggruppa gli indici dei sample per classe
class_indices = defaultdict(list)
for idx, sample in enumerate(test_dataset_windows):
    label = sample[2]  # Assumendo che sample = (window, label)
    class_indices[label].append(idx)

# Seleziona il 10% per ciascuna classe
val_idx = []
test_idx = []

for label, indices in class_indices.items():
    n_val = max(1, int(len(indices) * 0.1))  # almeno 1 se ci sono pochi
    permuted = np.random.permutation(indices)
    val_idx.extend(permuted[:n_val])
    test_idx.extend(permuted[n_val:])'''

# Crea i dataset
val_dataset = [test_dataset_windows[i] for i in val_idx]
test_dataset = [test_dataset_windows[i] for i in test_idx]

print(f"Numero di esempi nel test set: {len(test_dataset)}")

# 4) Creare un'istanza della classe HSI_Dataset per il test set
hsi_test_dataset = HSI_Dataset(test_dataset)
hsi_val_dataset = HSI_Dataset(val_dataset)

print(f"Dimensione totale del test set HSI: {len(hsi_test_dataset)}")

# 5) (Opzionale) Creare un DataLoader per il test set
# Solitamente non si mescolano i dati nel test set
test_batch_size = 32
test_shuffle = False

test_dataloader = DataLoader(hsi_test_dataset, batch_size=test_batch_size, shuffle=test_shuffle)
val_dataloader = DataLoader(hsi_val_dataset, batch_size=test_batch_size, shuffle=test_shuffle)

print(f"Numero di batch nel DataLoader per il test set: {len(test_dataloader)}")

# Ora hai il tuo test set pronto per essere utilizzato per valutare il modello.
# Puoi iterare su test_dataloader per ottenere batch di dati e label per il testing.

In [ ]:
!mkdir /content/{name}/dataset

In [ ]:
output_path = "/content/" + name + "/dataset/labels.txt"
with open(output_path, "w") as f:
  f.write(f"Distribuzione labels: {labels_recap}\n")

In [ ]:
torch.save(hsi_train_dataset.data_windows, "/content/"+ name +"/dataset/hsi_train_dataset.pt")
torch.save(hsi_val_dataset.data_windows, "/content/"+ name +"/dataset/hsi_val_dataset.pt")
torch.save(hsi_test_dataset.data_windows, "/content/"+ name +"/dataset/hsi_test_dataset.pt")

## Train

In [ ]:
vit = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
)

In [ ]:
log_dir = "/content/"+name+"/logs"
csv_logger = CSVLogger(save_dir=log_dir, name="")

In [ ]:
vitgatr_mnist = ViTGATr_LIGHT(
    model = vit,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.0, patience=20, verbose=False, mode="min")
'''checkpoint_callback = ModelCheckpoint(
    dirpath="/content/"+ name + "/checkpoints/",         # directory dove salvare
    filename="hci",  # nome file
    save_top_k=1,                   # salva solo il migliore (o tutti)
    monitor="loss",            # metrica da monitorare
    mode="min",                    # "min" o "max" a seconda della metrica
)'''
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci-{epoch:02d}-{accuracy:.3f}",
    monitor="accuracy_overall",
    mode="max",
    save_top_k=-1,   # salva tutti quelli che superano la soglia
)

checkpoint_loss = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci",
    monitor="val_loss",
    mode="min",
    save_top_k=1
)


In [ ]:
trainer_mnist = Trainer(logger=csv_logger, max_epochs=300, callbacks=[checkpoint_callback, checkpoint_loss, early_stop_callback])
trainer_mnist.fit(vitgatr_mnist, train_dataloader, val_dataloader)

## Test

In [ ]:
from fvcore.nn import parameter_count
params = parameter_count(vitgatr_mnist)

print(f"Params: {params[''] / 1e6:.2f} M")

In [ ]:
trainer_mnist.test(vitgatr_mnist, dataloaders=test_dataloader, verbose=False)
results_last = vitgatr_mnist.test_results
print(results_last)

accuracies_last = vitgatr_mnist.test_per_class_accuracies

In [ ]:
vit_l = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
)

In [ ]:
vitgatr_mnist_l = ViTGATr_LIGHT.load_from_checkpoint(
    "/content/" + name + "/checkpoints/hci.ckpt",
    #"/content/test-ip-16-07-N1/checkpoints/hci-epoch=221-accuracy=0.000.ckpt",
    model = vit_l,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

In [ ]:
trainer_mnist.test(vitgatr_mnist_l, dataloaders=test_dataloader, verbose=False)
results_loaded = vitgatr_mnist_l.test_results
print(results_loaded)

accuracies_loaded = vitgatr_mnist_l.test_per_class_accuracies

In [ ]:
output_path = "/content/" + name + "/results.txt"
with open(output_path, "w") as f:
    f.write(f"Params: {params[''] / 1e6:.3f} M\n\n")
    f.write(f"Results last\n")
    f.write(f"Accuracy Overall: {results_last[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_last[1]:.4f}\n")
    f.write(f"Kappa: {results_last[2]:.4f}\n\n")
    f.write(f"Results loaded\n")
    f.write(f"Accuracy Overall: {results_loaded[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_loaded[1]:.4f}\n")
    f.write(f"Kappa: {results_loaded[2]:.4f}\n")

    for i, acc in enumerate(accuracies_loaded):
        f.write(f"Classe {i} ({vitgatr_mnist_l.class_names[i]}): {acc:.4f}\n")

    for i, acc in enumerate(accuracies_last):
        f.write(f"Classe {i} ({vitgatr_mnist.class_names[i]}): {acc:.4f}\n")

## Download

In [ ]:
!cp -r /content/{name} /content/drive/MyDrive/tesi

In [ ]:
loop_counter = 13

# Loop tre canali

In [ ]:
name = f"test-{pre_name}-{date_str}-N{loop_counter}"
loop_counter += 1

In [ ]:
!mkdir /content/{name}

In [ ]:
parameters = dict({
    # Pre processing
    "batch_size": 8,
    "window_size": 16,
    "pca_channels": 16,

    # Vit
    "in_channels": 1,
    "out_channels": 16,
    "hidden_dim": 24,
    "n_heads": 8,
    "positional_dim": 120,
    "mv_in_channels": 32 + 32 + 16,
    "blocks": 1,
    "dropout_gatr": 0.2,
    "dropout_final": 0.2,
    "stride": 4,
    "type_embedding": "patch",

    # Training
    "lr": 1e-3,

    "note:": "PCA 16, PCA 16 + ICA, PCA 16 + ICA",
})

In [ ]:
# Converto il dizionario in stringa formattata
text = ""
for k, v in parameters.items():
    text += f"{k}: {v}\n"

# Salvo su file
with open("/content/" + name + "/parameters.txt", "w") as f:
    f.write(text)


## Dataset

In [ ]:
window_size = parameters["window_size"]
batch_size = parameters["batch_size"]
pca_channels = parameters["pca_channels"]
n_classes = n_classes

In [ ]:
'''

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_reshaped)

# Applica la PCA per ridurre i canali a 30
n_components = 16
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(data_scaled)

n_components_ica = pca_channels
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica = ica.fit_transform(data_pca_2)  # shape: (145*145, 16)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced = data_pca.reshape(data.shape[0], data.shape[1], n_components)
data_reduced = data_reduced.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced.shape}")'''
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_reshaped)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = 16
pca = PCA(n_components=n_components_pca)
data_pca = pca.fit_transform(data_scaled)  # shape: (145*145, 48)

'''
# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = 16
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica = ica.fit_transform(data_pca)  # shape: (145*145, 16)
'''
# Rimodella i dati ridotti nella forma spaziale originale
data_reduced = data_pca.reshape(data.shape[0], data.shape[1], n_components_pca)
data_reduced = data_reduced.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced.shape}")

Forma originale dei dati: (145, 145, 200)
Forma dopo PCA + ICA: (16, 145, 145)


In [ ]:
'''

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped_2 = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled_2 = scaler.fit_transform(data_reshaped_2)

# Applica la PCA per ridurre i canali a 30
n_components = 48
pca = PCA(n_components=n_components)
data_pca_2 = pca.fit_transform(data_scaled_2)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced_2 = data_pca_2.reshape(data.shape[0], data.shape[1], n_components)
data_reduced_2 = data_reduced_2.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced_2.shape}")'''
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped_2 = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled_2 = scaler.fit_transform(data_reshaped_2)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = 16
pca = PCA(n_components=n_components_pca)
data_pca_2 = pca.fit_transform(data_scaled_2)  # shape: (145*145, 48)

# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = 16
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica_2 = ica.fit_transform(data_pca_2)  # shape: (145*145, 16)

# Rimodella i dati ridotti nella forma spaziale originale
data_reduced_2 = data_ica_2.reshape(data.shape[0], data.shape[1], n_components_ica)
data_reduced_2 = data_reduced_2.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced_2.shape}")


Forma originale dei dati: (145, 145, 200)
Forma dopo PCA + ICA: (16, 145, 145)


In [ ]:
'''

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
# Da (altezza, larghezza, canali) a (altezza * larghezza, canali)
data_reshaped_3 = data.reshape(-1, data.shape[-1])

# Standardizza i dati (importante per la PCA)
scaler = StandardScaler()
data_scaled_3 = scaler.fit_transform(data_reshaped_3)

# Applica la PCA per ridurre i canali a 30
n_components = 64
pca = PCA(n_components=n_components)
data_pca_3 = pca.fit_transform(data_scaled_3)

# Rimodella i dati ridotti nella forma spaziale originale
# Da (altezza * larghezza, nuovi_canali) a (altezza, larghezza, nuovi_canali)
data_reduced_3 = data_pca_3.reshape(data.shape[0], data.shape[1], n_components)
data_reduced_3 = data_reduced_3.transpose(2, 0, 1)

print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dei dati dopo la riduzione PCA: {data_reduced_3.shape}")'''
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from skimage.morphology import disk
from skimage.morphology import erosion, dilation

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped_3 = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled_3 = scaler.fit_transform(data_reshaped_3)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = 16 * 4
pca = PCA(n_components=n_components_pca)
data_pca_3 = pca.fit_transform(data_scaled_3)  # shape: (145*145, 48)

'''
N = 4
scales = [1, 2, 3, 4]  # raggi degli elementi strutturanti

dmp_features = []

H, W, B = data.shape
data_pca_3 = data_pca_3.reshape(H, W, -1)

# Loop su ciascuna banda
for b in range(data_pca_3.shape[-1]):
    band = data_pca_3[:, :, b]

    band_features = []
    for s in scales:
        selem = disk(s)

        # Opening = dilation(erosion)
        opened = dilation(erosion(band, selem), selem)

        # Closing = erosion(dilation)
        closed = erosion(dilation(band, selem), selem)

        # Profilo differenziale = |closed - opened|
        dmp = np.abs(closed - opened)

        band_features.append(dmp)

    # Stack delle feature per questa banda: shape (4,4,N)
    band_features = np.stack(band_features, axis=-1)
    dmp_features.append(band_features)

# Concatenazione finale su tutte le bande
# Risultato: (4,4,16*N)
dmp_features = np.concatenate(dmp_features, axis=-1)

data_reduced_3 = dmp_features.transpose(2, 0, 1)

print("Output shape:", data_reduced_3.shape)  # (4,4,64)
'''

# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = 16 * 4
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica_3 = ica.fit_transform(data_pca_3)  # shape: (145*145, 16)

# Rimodella i dati ridotti nella forma spaziale originale
data_reduced_3 = data_ica_3.reshape(data.shape[0], data.shape[1], n_components_ica)
data_reduced_3 = data_reduced_3.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced_3.shape}")


Forma originale dei dati: (145, 145, 200)
Forma dopo PCA + ICA: (64, 145, 145)


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [ ]:
data_reduced = torch.from_numpy(data_reduced)
data_reduced_2 = torch.from_numpy(data_reduced_2)
data_reduced_3 = torch.from_numpy(data_reduced_3)
labels = torch.from_numpy(gt)

In [ ]:
tot_labels = 0
for row in labels:
  for column in row:
    if column != 0:
      tot_labels += 1
print("Numero totale di labels: ", tot_labels)
print("10% di tutti i labels: ", round(tot_labels / 100 * 10))

Numero totale di labels:  10249
10% di tutti i labels:  1025


In [ ]:
import random
if ten_per and global_ten_per:
    import numpy as np
    # 1) Calcolare i labels diversi da 0
    # Ottieni gli indici dei labels diversi da 0
    non_zero_labels_indices = np.where(labels.numpy() != 0)

    # Combina gli indici di riga e colonna in una lista di coordinate (riga, colonna)
    non_zero_labels_coords = list(zip(non_zero_labels_indices[0], non_zero_labels_indices[1]))

    print(f"Numero totale di labels diversi da 0: {len(non_zero_labels_coords)}")

    # 2) Prendere il 10% dei labels diversi da 10 in modo randomico
    import random

    # Calcola il 10% del numero totale di labels diversi da 0
    num_samples = int(len(non_zero_labels_coords) * 0.1)

    # Seleziona randomicamente gli indici
    random_non_zero_labels_coords = random.sample(non_zero_labels_coords, num_samples)

    unique_classes = torch.unique(labels[labels != 0])
    selected_coords_set = set(random_non_zero_labels_coords)

    for cls in unique_classes.tolist():
        # Trova tutti i pixel appartenenti a quella classe
        cls_indices = np.where(labels.numpy() == cls)
        cls_coords = list(zip(cls_indices[0], cls_indices[1]))

        # Filtra quelli già selezionati
        already_selected = [(r, c) for (r, c) in cls_coords if (r, c) in selected_coords_set]

        if len(already_selected) < 3:
            # Quanti ne mancano per arrivare a 3
            needed = 3 - len(already_selected)
            # Pixel ancora disponibili della classe
            remaining = list(set(cls_coords) - selected_coords_set)
            # Seleziona in modo randomico quelli mancanti
            extra = random.sample(remaining, min(needed, len(remaining)))
            selected_coords_set.update(extra)


    mask = torch.zeros_like(labels, dtype=torch.bool)

    # Imposta a True i pixel selezionati casualmente
    for row, col in selected_coords_set:
      mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero di True nella maschera:", mask.sum().item())


if ten_per and not global_ten_per:
    labels_np = labels.numpy()
    unique_labels = np.unique(labels_np)

    mask = torch.zeros_like(labels, dtype=torch.bool)

    for label in unique_labels:
        if label == 0:  # oppure `if label == 10` se intendevi escludere la classe 10
            continue

        # Trova le coordinate dei pixel con questa classe
        indices = np.where(labels_np == label)
        coords = list(zip(indices[0], indices[1]))

        # Calcola quanti pixel prendere
        num_samples = max(1, int(len(coords) * 0.1))  # almeno 1 se pochi pixel

        if len(coords) > 0:
            sampled_coords = random.sample(coords, num_samples)
            for row, col in sampled_coords:
                mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero di True nella maschera:", mask.sum().item())

Numero totale di labels diversi da 0: 10249
Forma della maschera: torch.Size([145, 145])
Numero di True nella maschera: 1026


In [ ]:
import random

if not ten_per:
    # Supponiamo che labels sia un tensore PyTorch
    labels_np = labels.numpy()

    # Ottieni le classi uniche diverse da 0
    unique_classes = np.unique(labels_np)
    unique_classes = unique_classes[unique_classes != 0]

    # Crea una maschera vuota
    mask = torch.zeros_like(labels, dtype=torch.bool)

    for cls in unique_classes:
        # Trova gli indici (riga, colonna) dei pixel di questa classe
        cls_indices = np.where(labels_np == cls)
        cls_coords = list(zip(cls_indices[0], cls_indices[1]))

        # Se ci sono meno di 100 pixel in quella classe, prendili tutti
        n = min(n_samples, len(cls_coords))

        # Seleziona randomicamente fino a 100 pixel
        selected_coords = random.sample(cls_coords, n)

        # Imposta a True nella maschera
        for row, col in selected_coords:
            mask[row, col] = True

    print("Forma della maschera:", mask.shape)
    print("Numero totale di True nella maschera:", mask.sum().item())


In [ ]:
labels_recap = np.zeros(n_classes)

final_labels = labels * mask
for row in final_labels:
  for column in row:
    if column != 0:
      labels_recap[column - 1] += 1

print(labels_recap)


[  5. 145.  82.  27.  55.  66.   3.  55.   4.  94. 234.  53.  17. 126.
  44.  16.]


In [ ]:
import torch
import random

class HSI_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_windows, augmented = False):
        """
        Args:
            data_windows (list): Lista di tuple (finestra, label) generata da create_dataset.
        """
        self.data_windows = data_windows
        self.augmented = augmented

    def __len__(self):
        return len(self.data_windows)

    def __getitem__(self, idx):

        window, window_2, window_3, label = self.data_windows[idx]
        # Converti la finestra in float32 per l'input del modello
        window = window.float()
        window_2 = window_2.float()
        window_3 = window_3.float()

        if self.augmented:
            window, window_2, window_3 = self.apply_augmentation(window, window_2, window_3)

        # Converti il label in LongTensor (per le funzioni di loss come CrossEntropyLoss)
        label = torch.tensor(label, dtype=torch.long)
        return [window, window_2, window_3], label

    def apply_augmentation(self, x1, x2, x3):
        # x1, x2: (C, H, W)
        if random.random() < 0.5:
            x1 = torch.flip(x1, dims=[1])  # Vertical flip
            x2 = torch.flip(x2, dims=[1])
            x3 = torch.flip(x3, dims=[1])
        if random.random() < 0.5:
            x1 = torch.flip(x1, dims=[2])  # Horizontal flip
            x2 = torch.flip(x2, dims=[2])
            x3 = torch.flip(x3, dims=[2])
        rot_k = random.choice([0, 1, 2, 3])  # 0°, 90°, 180°, 270°
        x1 = torch.rot90(x1, k=rot_k, dims=[1, 2])
        x2 = torch.rot90(x2, k=rot_k, dims=[1, 2])
        x3 = torch.rot90(x3, k=rot_k, dims=[1, 2])
        return x1, x2, x3


In [ ]:
# prompt: Ora voglio creare un dataset composot di "immagini" e label:
# 1) Le immagini sono una finistra sull'immagine originale di dimensioni 16x16
# 2) Queste finestre sono ottenute per ognuno dei pixel che sono true in mask, e rispetto alla finestra questi devono trovarsi in posizione (8,8)
# 3) Se la finestra esce fuori dall'immagine impostare tutti i pixel e tutti canali fuori dall'immagine a 0
# 4) Alla finestra deve essere associato il label corrispondente (nel dataset si accede con x alla finestra e a y al label)

def create_dataset(image, image_2, image_3, mask, window_size=16):
    """
    Crea un dataset di finestre di immagini centrate sui pixel della maschera.

    Args:
        image (torch.Tensor): L'immagine iperspettrale ridotta (H, W, C).
        mask (torch.Tensor): La maschera booleana (H, W).
        window_size (int): La dimensione della finestra (window_size x window_size).

    Returns:
        list: Una lista di tuple (finestra, label).
    """
    dataset = []
    c, h, w = image.shape
    pad = window_size // 2  # Padding necessario per centrare il pixel

    # Applica un padding all'immagine per gestire i bordi
    # Il padding è necessario su altezza e larghezza, e su tutti i canali (si usa 0 per i canali)
    padded_image = torch.nn.functional.pad(image, (pad, pad, pad, pad), "replicate")
    padded_image_2 = torch.nn.functional.pad(image_2, (pad, pad, pad, pad), "replicate")
    padded_image_3 = torch.nn.functional.pad(image_3, (pad, pad, pad, pad), "replicate")
    print("Padded image shape: ", padded_image.shape)
    print("Padded image shape2: ", padded_image_2.shape)

    # Trova le coordinate (riga, colonna) dei pixel True nella maschera
    rows, cols = torch.where(mask)

    for r, c in zip(rows, cols):
        # Calcola le coordinate della finestra nell'immagine con padding
        # Il pixel di interesse (r, c) nell'immagine originale corrisponde a (r + pad, c + pad) nell'immagine con padding
        start_row = r + pad - pad
        end_row = r + pad + pad
        start_col = c + pad - pad
        end_col = c + pad + pad

        # Estrai la finestra dall'immagine con padding
        window = padded_image[:, start_row:end_row, start_col:end_col]
        window_2 = padded_image_2[:, start_row:end_row, start_col:end_col]
        window_3 = padded_image_3[:, start_row:end_row, start_col:end_col]

        # Ottieni il label dal tensore originale delle etichette
        label = labels[r, c].item() - 1

        # Aggiungi la finestra e il label al dataset
        dataset.append((window, window_2, window_3, label))

    return dataset

# Crea il dataset utilizzando l'immagine ridotta e la maschera creata in precedenza
dataset = create_dataset(data_reduced, data_reduced_2, data_reduced_3, mask, window_size=window_size)

print(f"Numero di esempi nel dataset: {len(dataset)}")

# Esempio di accesso al primo elemento del dataset
if len(dataset) > 0:
    first_window, second_window, third_window, first_label = dataset[0]
    print(f"Forma della prima finestra: {first_window.shape}")
    print(f"Label del primo esempio: {first_label}")


Padded image shape:  torch.Size([16, 161, 161])
Padded image shape2:  torch.Size([16, 161, 161])
Numero di esempi nel dataset: 1026
Forma della prima finestra: torch.Size([16, 16, 16])
Label del primo esempio: 2


In [ ]:
'''if pre_name == 'ip':
    to_push = []
    for element in dataset:
        label = element[3]
        if label == 0 or label == 6 or label == 8:
          window1 = element[0]
          window2 = element[1]
          window3 = element[2]

          rot90_w1 = torch.rot90(window1, k=-1, dims=[1, 2])
          rot90_w2 = torch.rot90(window2, k=-1, dims=[1, 2])
          rot90_w3 = torch.rot90(window3, k=-1, dims=[1, 2])

          rotm90_w1 = torch.rot90(window1, k=1, dims=[1, 2])
          rotm90_w2 = torch.rot90(window2, k=1, dims=[1, 2])
          rotm90_w3 = torch.rot90(window3, k=1, dims=[1, 2])

          flipV_w1 = torch.flip(window1, dims=[1, 2])
          flipV_w2 = torch.flip(window2, dims=[1, 2])
          flipV_w3 = torch.flip(window3, dims=[1, 2])

          to_push.append((rot90_w1, rot90_w2, rot90_w3, label))
          to_push.append((rotm90_w1, rotm90_w2, rotm90_w3, label))
          to_push.append((flipV_w1, flipV_w2, flipV_w3, label))

    for element in to_push:
        dataset.append(element)

    labels_recap = np.zeros(n_classes)

    for element in dataset:
          labels_recap[element[3]] += 1

    print(labels_recap)'''

"if pre_name == 'ip':\n    to_push = []\n    for element in dataset:\n        label = element[3]\n        if label == 0 or label == 6 or label == 8:\n          window1 = element[0]\n          window2 = element[1]\n          window3 = element[2]\n\n          rot90_w1 = torch.rot90(window1, k=-1, dims=[1, 2])\n          rot90_w2 = torch.rot90(window2, k=-1, dims=[1, 2])\n          rot90_w3 = torch.rot90(window3, k=-1, dims=[1, 2])\n\n          rotm90_w1 = torch.rot90(window1, k=1, dims=[1, 2])\n          rotm90_w2 = torch.rot90(window2, k=1, dims=[1, 2])\n          rotm90_w3 = torch.rot90(window3, k=1, dims=[1, 2])\n\n          flipV_w1 = torch.flip(window1, dims=[1, 2])\n          flipV_w2 = torch.flip(window2, dims=[1, 2])\n          flipV_w3 = torch.flip(window3, dims=[1, 2])\n\n          to_push.append((rot90_w1, rot90_w2, rot90_w3, label))\n          to_push.append((rotm90_w1, rotm90_w2, rotm90_w3, label))\n          to_push.append((flipV_w1, flipV_w2, flipV_w3, label))\n\n    for e

In [ ]:
# prompt: tramite dataset vorrei creare la classe dataset e dataloader

from torch.utils.data import Dataset, DataLoader

# Crea un'istanza del dataset
hsi_train_dataset = HSI_Dataset(dataset, augmented=True)

print(f"Dimensione totale del dataset HSI: {len(hsi_train_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = True  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
train_dataloader = DataLoader(hsi_train_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(train_dataloader)}")

# Esempio di iterazione attraverso il DataLoader
# for batch_windows, batch_labels in dataloader:
#     print(f"Forma del batch di finestre: {batch_windows.shape}")
#     print(f"Forma del batch di labels: {batch_labels.shape}")
#     break # Esci dopo il primo batch per dimostrazione



Dimensione totale del dataset HSI: 1026
Numero di batch nel DataLoader: 129


In [ ]:
# prompt: ora però devo creare il test set con gli altri pixel che non ho usato

# Creazione del test set con i pixel rimanenti

# 1) Identificare i pixel che NON sono stati usati nel training set (maschera è False)
# Creiamo una maschera per il test set, che è l'inverso della maschera di training
test_mask = ~mask

print("Forma della maschera per il test set:", test_mask.shape)
print("Numero di True nella maschera del test set:", test_mask.sum().item())

# 2) Filtrare i pixel del test_mask per includere solo quelli con label > 0
# Vogliamo che il test set contenga solo pixel con classi effettive, non lo sfondo (label 0)
# Quindi, creiamo una maschera combinata dove sia test_mask che (labels > 0) sono True
test_mask_filtered = test_mask & (labels > 0)

print("Forma della maschera per il test set filtrata (solo pixel con label > 0):", test_mask_filtered.shape)
print("Numero di True nella maschera filtrata:", test_mask_filtered.sum().item())


# 3) Utilizzare la funzione create_dataset per creare il test set
# Passiamo l'immagine ridotta e la nuova maschera filtrata
test_dataset_windows = create_dataset(data_reduced, data_reduced_2, data_reduced_3, test_mask_filtered, window_size=window_size)

n_val = int(len(test_dataset_windows) * 0.1)
indices = np.random.permutation(len(test_dataset_windows))

'''val_idx = indices[:n_val]
test_idx = indices[n_val:]

val_dataset = [test_dataset_windows[i] for i in val_idx]
test_dataset = [test_dataset_windows[i] for i in test_idx]'''

from collections import defaultdict

# Raggruppa gli indici dei sample per classe
class_indices = defaultdict(list)
for idx, sample in enumerate(test_dataset_windows):
    label = sample[3]  # Assumendo che sample = (window, label)
    class_indices[label].append(idx)

# Seleziona il 10% per ciascuna classe
val_idx = []
test_idx = []

if False:
  for label, indices in class_indices.items():
      n_val = min(100, len(indices))  # massimo 10 elementi, o meno se la classe è piccola
      permuted = np.random.permutation(indices)
      val_idx.extend(permuted[:n_val])     # prendi i 10 (o meno) per la validazione
      test_idx.extend(permuted[n_val:])    # il resto va nel test

if True:
  for label, indices in class_indices.items():
      n_val = max(1, int(len(indices) * 0.1))  # almeno 1 se ci sono pochi
      permuted = np.random.permutation(indices)
      val_idx.extend(permuted[:n_val])
      test_idx.extend(permuted[n_val:])


# Crea i dataset
val_dataset = [test_dataset_windows[i] for i in val_idx]
test_dataset = [test_dataset_windows[i] for i in test_idx]

print(f"Numero di esempi nel test set: {len(test_dataset)}")

# 4) Creare un'istanza della classe HSI_Dataset per il test set
hsi_test_dataset = HSI_Dataset(test_dataset)
hsi_val_dataset = HSI_Dataset(val_dataset)

print(f"Dimensione totale del test set HSI: {len(hsi_test_dataset)}")

# 5) (Opzionale) Creare un DataLoader per il test set
# Solitamente non si mescolano i dati nel test set
test_batch_size = 32
test_shuffle = False

test_dataloader = DataLoader(hsi_test_dataset, batch_size=test_batch_size, shuffle=test_shuffle)
val_dataloader = DataLoader(hsi_val_dataset, batch_size=test_batch_size, shuffle=test_shuffle)

print(f"Numero di batch nel DataLoader per il test set: {len(test_dataloader)}")

# Ora hai il tuo test set pronto per essere utilizzato per valutare il modello.
# Puoi iterare su test_dataloader per ottenere batch di dati e label per il testing.

Forma della maschera per il test set: torch.Size([145, 145])
Numero di True nella maschera del test set: 19999
Forma della maschera per il test set filtrata (solo pixel con label > 0): torch.Size([145, 145])
Numero di True nella maschera filtrata: 9223
Padded image shape:  torch.Size([16, 161, 161])
Padded image shape2:  torch.Size([16, 161, 161])
Numero di esempi nel test set: 8308
Dimensione totale del test set HSI: 8308
Numero di batch nel DataLoader per il test set: 260


In [ ]:
print(f"Dimensione totale del test set HSI: {len(hsi_val_dataset)}")

Dimensione totale del test set HSI: 915


In [ ]:
!mkdir /content/{name}/dataset

In [ ]:
output_path = "/content/" + name + "/dataset/labels.txt"
with open(output_path, "w") as f:
  f.write(f"Distribuzione labels: {labels_recap}\n")

In [ ]:
torch.save(hsi_train_dataset.data_windows, "/content/"+ name +"/dataset/hsi_train_dataset.pt")
torch.save(hsi_val_dataset.data_windows, "/content/"+ name +"/dataset/hsi_val_dataset.pt")
torch.save(hsi_test_dataset.data_windows, "/content/"+ name +"/dataset/hsi_test_dataset.pt")

## Train

In [ ]:
vit = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    dropout_final = parameters["dropout_final"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
    window_size = parameters['window_size'],
)

In [ ]:
vit = vit.to(device)

In [ ]:
a, b = next(iter(train_dataloader))

In [ ]:
print(a[0].shape)
print(a[1].shape)
print(a[2].shape)

torch.Size([8, 16, 16, 16])
torch.Size([8, 16, 16, 16])
torch.Size([8, 64, 16, 16])


In [ ]:
a1 = torch.rand(1, 32, 16, 16).to(device)
a2 = torch.rand(1, 32, 16, 16).to(device)
a3 = torch.rand(1, 64, 16, 16).to(device)
l = [a1, a2, a3]

In [ ]:
a1 = torch.rand(1, 16, 16, 16).to(device)
a2 = torch.rand(1, 16, 16, 16).to(device)
a3 = torch.rand(1, 64, 16, 16).to(device)
l = [a1, a2, a3]
vit(l)

torch.Size([1, 16, 48, 16])


tensor([[  0.6726,   2.2457,  -4.7027,  -2.4555,   2.8650,   0.9654,   0.4639,
          -0.3162,   0.3607,  -5.2915,   1.8772,   4.7903,   4.5232, -18.2489,
           2.4781,  -1.5143]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [ ]:
!pip install ptflops

In [ ]:
sum(p.numel() for p in vit.parameters())

106014

In [ ]:
!pip install thop

In [ ]:
!pip install fvcore

In [ ]:
from fvcore.nn import FlopCountAnalysis, parameter_count

input_data = (l,)
flops = FlopCountAnalysis(vit, input_data)
params = parameter_count(vit)
print("FLOPs:", flops.total(), "FLOPs")
print("Params:", params[""])


torch.Size([1, 16, 80, 16])


gatr.gatr.linear_out.mvs2s, gatr.gatr.linear_out.s2s


FLOPs: 596205932.0 FLOPs
Params: 106014


In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

# Assumiamo che il tuo modello e input siano:
# vit = modello
# l = lista di tensori di input (es. [a1, a2, a3])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
vit = vit.to(device)
l = [x.to(device) for x in l]  # sposta gli input sulla stessa device

# Funzione helper per stimare MACs da FLOPs
def flops_to_macs(flops):
    return flops / 2

# Profiling reale con torch.profiler
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    with_stack=False
) as prof:
    with record_function("model_inference"):
        out = vit(l)  # forward reale

# Stampa tabella con le operazioni più costose
print(prof.key_averages().table(
    sort_by="cuda_time_total", row_limit=20
))

# Stima FLOPs e MACs per le operazioni chiave
total_flops = 0
for evt in prof.key_averages():
    # stimiamo le FLOPs principali: linear, bmm, einsum
    if 'linear' in evt.key.lower() or 'bmm' in evt.key.lower() or 'einsum' in evt.key.lower():
        # esempio semplice: moltiplichiamo numel output * 2 come FLOPs
        # puoi personalizzare per ciascun tipo di operazione
        if hasattr(evt, 'shape') and evt.shape is not None:
            flops = 2 * torch.prod(torch.tensor(evt.shape))
        else:
            flops = evt.cpu_time_total * 1e3  # fallback se shape non disponibile
        total_flops += flops

total_macs = flops_to_macs(total_flops)
print(f"Stima totale FLOPs: {total_flops:,.0f}")
print(f"Stima totale MACs: {total_macs:,.0f}")


torch.Size([1, 16, 48, 16])
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     120.486ms        98.62%     120.486ms     120.486ms             1  
                                        model_inference        18.45%      24.494ms        99.99%     132.739ms     132.739ms       0.000us         0.00%       1.684ms       1.684

In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

inputs = (l,)  # o un batch reale

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        vit(*inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))


torch.Size([1, 16, 48, 16])
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  model_inference        38.08%      35.746ms       100.00%      93.878ms      93.878ms             1  
                                     aten::einsum         1.96%       1.838ms        19.14%      17.969ms     816.759us            22  
                                     aten::linear         0.37%     348.049us        13.35%      12.530ms     432.060us            29  
                                    aten::reshape         0.76%     717.471us         7.94%       7.452ms      59.147us           126  
                    

In [ ]:
from thop import profile

macs, params = profile(vit, inputs=(l,))
print(f"MACs: {macs} G, Params: {params}")


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
torch.Size([1, 16, 48, 16])
MACs: 120256.0 G, Params: 7870.0


In [ ]:
log_dir = "/content/"+name+"/logs"
csv_logger = CSVLogger(save_dir=log_dir, name="")

In [ ]:
vitgatr_mnist = ViTGATr_LIGHT(
    model = vit,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

Class weights: tensor([5.0000, 0.5000, 0.7820, 2.3750, 1.1659, 0.9716, 5.0000, 1.1659, 5.0000,
        0.6822, 0.5000, 1.2099, 3.7721, 0.5089, 1.4574, 4.0078])
Weight for trees (classe 0): tensor(5.)


In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.0, patience=40, verbose=False, mode="min")
'''checkpoint_callback = ModelCheckpoint(
    dirpath="/content/"+ name + "/checkpoints/",         # directory dove salvare
    filename="hci",  # nome file
    save_top_k=1,                   # salva solo il migliore (o tutti)
    monitor="loss",            # metrica da monitorare
    mode="min",                    # "min" o "max" a seconda della metrica
)'''
'''checkpoint_callback = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci-{epoch:02d}-{accuracy:.3f}",
    monitor="accuracy_overall",
    mode="max",
    save_top_k=-1,   # salva tutti quelli che superano la soglia
)'''

checkpoint_loss = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci",
    monitor="val_loss",
    mode="min",
    save_top_k=1
)


In [ ]:
trainer_mnist = Trainer(logger=csv_logger, max_epochs=400, callbacks=[checkpoint_loss, early_stop_callback])
trainer_mnist.fit(vitgatr_mnist, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                    | Type                      | Params | Mode 
-------------------------------------------------------------------------------
0  | accuracy                | MulticlassAccuracy        | 0      | train
1  | train_accuracy          | MulticlassAccuracy        | 0      | train
2  | accuracy_overall        | MulticlassAccuracy        | 0      | train
3  | accuracy_avg            | MulticlassAccuracy        | 0      | train
4  | kappa                   | MulticlassCohenKappa      | 0      | train
5  | train_accuracy_overall  | MulticlassAccuracy        | 0      | train
6  | train_accuracy_avg   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

torch.Size([32, 16, 48, 16])
torch.Size([32, 16, 48, 16])
[Validation] Epoch: 0  Loss: 17.4363  Accuracy Overall: 0.0000  Accuracy Avg: 0.0000  Kappa: 0.0000


Training: |          | 0/? [00:00<?, ?it/s]

torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])


INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
torch.Size([8, 16, 48, 16])
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 598, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1011, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1055, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py", line 216, in run
    self.advance()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/

TypeError: object of type 'NoneType' has no len()

## Test

In [ ]:
from fvcore.nn import parameter_count
params = parameter_count(vitgatr_mnist)

print(f"Params: {params[''] / 1e6:.2f} M")

Params: 0.05 M


In [ ]:
trainer_mnist.test(vitgatr_mnist, dataloaders=test_dataloader, verbose=False)
results_last = vitgatr_mnist.test_results
print(results_last)

accuracies_last = vitgatr_mnist.test_per_class_accuracies

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[Test] accurracy 0.9858; accuracy avg 0.9945; kappa 0.9841
[Test] Accuracy per classe:
  Classe 0 (Brocoli_green_weeds_1): 1.0000
  Classe 1 (Brocoli_green_weeds_2): 1.0000
  Classe 2 (Fallow): 1.0000
  Classe 3 (Fallow_rough_plow): 1.0000
  Classe 4 (Fallow_smooth): 0.9910
  Classe 5 (Stubble): 1.0000
  Classe 6 (Celery): 1.0000
  Classe 7 (Grapes_untrained): 0.9539
  Classe 8 (Soil_vinyard_develop): 1.0000
  Classe 9 (Corn_senesced_green_weeds): 0.9955
  Classe 10 (Lettuce_romaine_4wk): 1.0000
  Classe 11 (Lettuce_romaine_5wk): 0.9994
  Classe 12 (Lettuce_romaine_6wk): 1.0000
  Classe 13 (Lettuce_romaine_7wk): 1.0000
  Classe 14 (Vinyard_untrained): 0.9729
  Classe 15 (Vinyard_vertical_trellis): 1.0000
[0.9857674837112427, 0.9945391416549683, 0.9841182231903076]


In [ ]:
vit_l = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
)

In [ ]:
vitgatr_mnist_l = ViTGATr_LIGHT.load_from_checkpoint(
    "/content/" + name + "/checkpoints/hci.ckpt",
    #"/content/hci.ckpt",
    model = vit_l,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

Class weights: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Weight for trees (classe 0): tensor(1.)


RuntimeError: Error(s) in loading state_dict for ViTGATr_LIGHT:
	Missing key(s) in state_dict: "vit.gatr.vectorizer.weight", "vit.gatr.vectorizer.bias". 

In [ ]:
import torch
from torch.utils.data import DataLoader

# Carica i dati salvati
train_windows = torch.load("/content/hsi_train_dataset.pt")
val_windows   = torch.load("/content/hsi_val_dataset.pt")
test_windows  = torch.load("/content/hsi_test_dataset.pt")

# Ricrea i dataset
hsi_train_dataset = HSI_Dataset(train_windows, augmented=True)
hsi_val_dataset   = HSI_Dataset(val_windows, augmented=False)
hsi_test_dataset  = HSI_Dataset(test_windows, augmented=False)

# Crea i DataLoader per usarli come prima
train_loader = DataLoader(hsi_train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(hsi_val_dataset, batch_size=64, shuffle=False)
test_loader  = DataLoader(hsi_test_dataset, batch_size=64, shuffle=False)


In [ ]:
trainer_mnist.test(vitgatr_mnist_l, dataloaders=val_dataloader, verbose=False)
results_loaded = vitgatr_mnist_l.test_results
print(results_loaded)

accuracies_loaded = vitgatr_mnist_l.test_per_class_accuracies

In [ ]:
class FullImageDataset(Dataset):
    def __init__(self, image, image_2, image_3, labels, window_size=16):
        """
        Crea tutte le finestre centrate sui pixel dell'immagine
        e scarta quelli con label == 0.
        """
        self.image = image
        self.image_2 = image_2
        self.image_3 = image_3
        self.labels = labels
        self.window_size = window_size
        self.pad = window_size // 2

        self.padded_image = F.pad(image, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.padded_image_2 = F.pad(image_2, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.padded_image_3 = F.pad(image_3, (self.pad, self.pad, self.pad, self.pad), "replicate")

        # Salva dimensioni
        _, self.H, self.W = image.shape

        # Costruisci lista di coordinate valide (label > 0)
        rows, cols = torch.where(labels > 0)
        self.coords = list(zip(rows.tolist(), cols.tolist()))

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, idx):
        r, c = self.coords[idx]
        start_row = r
        end_row = r + self.window_size
        start_col = c
        end_col = c + self.window_size

        window = self.padded_image[:, start_row:end_row, start_col:end_col].float()
        window_2 = self.padded_image_2[:, start_row:end_row, start_col:end_col].float()
        window_3 = self.padded_image_3[:, start_row:end_row, start_col:end_col].float()

        return [window, window_2, window_3], (r, c)



In [ ]:
full_dataset = FullImageDataset(data_reduced, data_reduced_2, data_reduced_3, labels, window_size=window_size)
loader_full = DataLoader(full_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch
import torch.nn.functional as F

# Supponiamo di avere le dimensioni originali
_, H, W = data_reduced.shape
pred_map = torch.zeros((H, W), dtype=torch.int64)  # 0 = sfondo

device = "cuda" if torch.cuda.is_available() else "cpu"
vitgatr_mnist_l.to(device)
vitgatr_mnist_l.eval()

with torch.no_grad():
    for windows, coords in loader_full:
        # windows è una lista di tensori: [window1_batch, window2_batch, window3_batch]
        windows = [w.to(device) for w in windows]

        # Forward
        outputs = vitgatr_mnist_l(windows)
        preds = torch.argmax(outputs.logits, dim=1).cpu()
        # coords è ora una lista di tuple (r, c)
        rows, cols = coords  # coords è una tuple di due tensori
        for i in range(len(rows)):
            r = rows[i].item()
            c = cols[i].item()
            pred_map[r, c] = preds[i] + 1  # +1 se vuoi preservare 0 come sfondo






In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

colors = [(0,0,0),(1,0,0),(0,1,0),(0,0,1),(1,1,0),(1,0,1),(0,1,1),(0.5,0.5,0.5),
                   (0.5,0,0),(0,0.5,0),(0,0,0.5),(0.75,0.25,0.5),(0.25,0.75,0.5),(0.5,0.25,0.75),
                   (0.75,0.5,0.25),(0.25,0.5,0.75),(0.6,0.4,0.2),(0.2,0.6,0.4)]

# pred_map: (H, W), valori 0..15
H, W = pred_map.shape

# Converte la mappa dei predetti in un'immagine RGB
pred_rgb = torch.zeros((H, W, 3), dtype=torch.float32)

# Associa i colori
for i, color in enumerate(colors):
    mask = pred_map == i
    for j in range(3):  # R, G, B
        pred_rgb[..., j][mask] = color[j]

# Visualizza l'immagine
plt.figure(figsize=(8, 8))
plt.imshow(pred_rgb.numpy())
plt.axis("off")
plt.show()



In [ ]:
plt.imsave("/content/pred_rgb.png", pred_rgb.numpy())


In [ ]:
output_path = "/content/" + name + "/results.txt"
with open(output_path, "w") as f:
    f.write(f"Params: {params[''] / 1e6:.3f} M\n\n")
    f.write(f"Results last\n")
    f.write(f"Accuracy Overall: {results_last[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_last[1]:.4f}\n")
    f.write(f"Kappa: {results_last[2]:.4f}\n\n")
    f.write(f"Results loaded\n")
    f.write(f"Accuracy Overall: {results_loaded[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_loaded[1]:.4f}\n")
    f.write(f"Kappa: {results_loaded[2]:.4f}\n")

    f.write(f"\nLast\n")
    for i, acc in enumerate(accuracies_last):
        f.write(f"Classe {i} ({vitgatr_mnist.class_names[i]}): {acc:.4f}\n")

    f.write(f"\nLoad\n")
    for i, acc in enumerate(accuracies_loaded):
        f.write(f"Classe {i} ({vitgatr_mnist_l.class_names[i]}): {acc:.4f}\n")


## Download

In [ ]:
!cp -r /content/{name} /content/drive/MyDrive/tesi

# Loop tre canali - Altri Dataset

In [ ]:
name = f"test-{pre_name}-{date_str}-N{loop_counter}"
loop_counter += 1

NameError: name 'pre_name' is not defined

In [ ]:
!mkdir /content/{name}

In [ ]:
parameters = dict({
    # Pre processing
    "batch_size": 64,
    "window_size": 16,
    "pca_channels": 16,

    # Vit
    "in_channels": 1,
    "out_channels": 16,
    "hidden_dim": 24,
    "n_heads": 8,
    "positional_dim": 128,
    "Hor": 16,
    "Ver": 16,
    "3D": 16,
    "mv_in_channels": 16 + 16 + 16,
    "blocks": 1,
    "dropout_gatr": 0.4,
    "dropout_final": 0.4,
    "stride": 4,
    "type_embedding": "patch",
    "dsm_embedding": 128,

    # Training
    "lr": 1e-3,

    "note:": "PCA 32, PCA 32 + ICA, PCA 64 + ICA",
})

In [ ]:
# Converto il dizionario in stringa formattata
text = ""
for k, v in parameters.items():
    text += f"{k}: {v}\n"

# Salvo su file
with open("/content/" + name + "/parameters.txt", "w") as f:
    f.write(text)


## Dataset

In [ ]:
window_size = parameters["window_size"]
batch_size = parameters["batch_size"]
pca_channels = parameters["pca_channels"]
n_classes = n_classes

In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_reshaped)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = parameters["Hor"]
pca = PCA(n_components=n_components_pca)
data_pca = pca.fit_transform(data_scaled)  # shape: (145*145, 48)

'''
# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = parameters["Hor"]
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica = ica.fit_transform(data_pca)  # shape: (145*145, 16)
'''
# Rimodella i dati ridotti nella forma spaziale originale
data_reduced = data_pca.reshape(data.shape[0], data.shape[1], n_components_pca)
data_reduced = data_reduced.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced.shape}")

Forma originale dei dati: (349, 1905, 144)
Forma dopo PCA + ICA: (32, 349, 1905)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped_2 = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled_2 = scaler.fit_transform(data_reshaped_2)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = parameters["Ver"]
pca = PCA(n_components=n_components_pca)
data_pca_2 = pca.fit_transform(data_scaled_2)  # shape: (145*145, 48)


# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = parameters["Ver"]
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica_2 = ica.fit_transform(data_pca_2)  # shape: (145*145, 16)


# Rimodella i dati ridotti nella forma spaziale originale
data_reduced_2 = data_ica_2.reshape(data.shape[0], data.shape[1], n_components_ica)
data_reduced_2 = data_reduced_2.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced_2.shape}")


Forma originale dei dati: (349, 1905, 144)
Forma dopo PCA + ICA: (32, 349, 1905)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from skimage.morphology import disk
from skimage.morphology import erosion, dilation

# Rimodella i dati per l'applicazione della PCA (flattening spaziale)
data_reshaped_3 = data.reshape(-1, data.shape[-1])  # shape: (145*145, 200)

# Standardizza i dati
scaler = StandardScaler()
data_scaled_3 = scaler.fit_transform(data_reshaped_3)  # shape: (145*145, 200)

# Applica PCA per ridurre i canali a 48
n_components_pca = parameters["3D"] * 4
pca = PCA(n_components=n_components_pca)
data_pca_3 = pca.fit_transform(data_scaled_3)  # shape: (145*145, 48)


# Applica ICA per ridurre ulteriormente a 16 componenti indipendenti
n_components_ica = parameters["3D"] * 4
ica = FastICA(n_components=n_components_ica, random_state=0)
data_ica_3 = ica.fit_transform(data_pca_3)  # shape: (145*145, 16)

# Rimodella i dati ridotti nella forma spaziale originale
data_reduced_3 = data_ica_3.reshape(data.shape[0], data.shape[1], n_components_ica)
data_reduced_3 = data_reduced_3.transpose(2, 0, 1)  # shape: (16, 145, 145)

# Stampa le forme
print(f"Forma originale dei dati: {data.shape}")
print(f"Forma dopo PCA + ICA: {data_reduced_3.shape}")


Forma originale dei dati: (349, 1905, 144)
Forma dopo PCA + ICA: (48, 349, 1905)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
data_reduced = torch.from_numpy(data_reduced)
data_reduced_2 = torch.from_numpy(data_reduced_2)
data_reduced_3 = torch.from_numpy(data_reduced_3)

In [ ]:
import torch
import random
import torchvision.transforms.functional as TF

class HSI_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_windows, augmented = False):
        """
        Args:
            data_windows (list): Lista di tuple (finestra, label) generata da create_dataset.
        """
        self.data_windows = data_windows
        self.augmented = augmented

    def __len__(self):
        return len(self.data_windows)

    def __getitem__(self, idx):

        window, window_2, window_3, dsm_window, label = self.data_windows[idx]
        # Converti la finestra in float32 per l'input del modello
        window = window.float()
        window_2 = window_2.float()
        window_3 = window_3.float()
        dsm_window = dsm_window.float()

        if self.augmented:
            window, window_2, window_3 = self.apply_augmentation(window, window_2, window_3)

        # Converti il label in LongTensor (per le funzioni di loss come CrossEntropyLoss)
        label = torch.tensor(label, dtype=torch.long)
        return [window, window_2, window_3, dsm_window], label

    def random_blur(x, p=0.5, kernel_size=3):
        if random.random() < p:
            x = x.unsqueeze(0)  # aggiungi batch dimension
            x = F.avg_pool2d(x, kernel_size, stride=1, padding=kernel_size//2)
            x = x.squeeze(0)
        return x

    def random_gamma(self, x, p=0.5, gamma_range=(0.8, 1.2)):
      if random.random() < p:
          gamma = random.uniform(*gamma_range)
          x = x ** gamma
      return x

    def random_speckle(self, x, p=0.5, scale=0.05):
        if random.random() < p:
            x = x + x * torch.randn_like(x) * scale
        return x

    def random_affine(self, x, max_translation=2):
        tx = random.randint(-max_translation, max_translation)
        ty = random.randint(-max_translation, max_translation)
        x = TF.affine(x, angle=0, translate=(tx, ty), scale=1.0, shear=0)
        return x

    def mixup(self, x1, x2, alpha=0.4):
        lam = np.random.beta(alpha, alpha)
        x = lam * x1 + (1-lam) * x2
        return x

    def random_band_dropout(self, x, p=0.1):
        C, H, W = x.shape
        for c in range(C):
            if random.random() < p:
                x[c, :, :] = 0
        return x


    def spectral_jittering(self, x, sigma=0.01):
        noise = torch.randn_like(x) * sigma
        return x + noise


    def random_holes(self, x, hole_size=(5, 5), n_holes=1, fill_value=0.0):
        """
        Inserisce dei 'buchi' (occlusioni) casuali nell'immagine iperspettrale.

        Args:
            x: tensor (C, H, W)
            hole_size: (h, w) dimensione del buco
            n_holes: numero di buchi da inserire
            fill_value: valore con cui riempire i buchi (0 = nero, oppure rumore)
        """
        C, H, W = x.shape
        h, w = hole_size

        for _ in range(n_holes):
            top = random.randint(0, H - h)
            left = random.randint(0, W - w)

            if isinstance(fill_value, str) and fill_value == "noise":
                # Rumore al posto del buco
                x[:, top:top+h, left:left+w] = torch.randn((C, h, w), device=x.device) * 0.1
            else:
                # Default: azzera
                x[:, top:top+h, left:left+w] = fill_value
        return x


    def apply_augmentation(self, x1, x2, x3):
        def augment(x):
            x = self.spectral_jittering(x)
            x = self.random_holes(x, hole_size=(3,3))
            x = torch.flip(x, dims=[1]) if random.random() < 0.5 else x
            x = torch.flip(x, dims=[2]) if random.random() < 0.5 else x
            x = torch.rot90(x, k=random.choice([0,1,2,3]), dims=[1,2])
            x = self.random_speckle(x)
            x = self.random_band_dropout(x, p=0.1)
            # eventualmente aggiungi random crop

            if random.random() < 0.2:
              displacement = torch.randn(2, 1, 1) * 0.02
              grid = torch.stack(torch.meshgrid(torch.linspace(-1,1,x.shape[1]), torch.linspace(-1,1,x.shape[2])), 0)
              grid = grid + displacement
              x = F.grid_sample(x.unsqueeze(0), grid.permute(1,2,0).unsqueeze(0), align_corners=True).squeeze(0)

            return x

        # Applica mixup tra finestre con probabilità 0.3
        if random.random() < 0.3:
            lam = np.random.beta(0.4, 0.4)
            x2 = lam * x1 + (1-lam) * x2

        return augment(x1), augment(x2), augment(x3)





In [ ]:
# prompt: Ora voglio creare un dataset composot di "immagini" e label:
# 1) Le immagini sono una finistra sull'immagine originale di dimensioni 16x16
# 2) Queste finestre sono ottenute per ognuno dei pixel che sono true in mask, e rispetto alla finestra questi devono trovarsi in posizione (8,8)
# 3) Se la finestra esce fuori dall'immagine impostare tutti i pixel e tutti canali fuori dall'immagine a 0
# 4) Alla finestra deve essere associato il label corrispondente (nel dataset si accede con x alla finestra e a y al label)

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset

class WindowDataset(Dataset):
    def __init__(self, image, image_2, image_3, i_dsm, mask, dense, window_size=16):
        self.pad = window_size // 2
        self.image = F.pad(image, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.image_2 = F.pad(image_2, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.image_3 = F.pad(image_3, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.dsm = F.pad(i_dsm, (self.pad, self.pad, self.pad, self.pad), "replicate")
        self.dense = dense
        self.window_size = window_size

        self.rows, self.cols = torch.where(mask)

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx, augmented = False):
        r, c = self.rows[idx], self.cols[idx]
        start_row = r
        end_row = r + self.window_size
        start_col = c
        end_col = c + self.window_size

        # padding automatico
        window = F.pad(self.image[:, start_row:end_row, start_col:end_col],
                      (0, max(0, end_col - self.image.shape[2]),
                        0, max(0, end_row - self.image.shape[1])), "constant", 0)
        window_2 = F.pad(self.image_2[:, start_row:end_row, start_col:end_col], (0,0,0,0), "constant", 0)
        window_3 = F.pad(self.image_3[:, start_row:end_row, start_col:end_col], (0,0,0,0), "constant", 0)
        dsm_window = F.pad(self.dsm[:, start_row:end_row, start_col:end_col], (0,0,0,0), "constant", 0)

        label = self.dense[r, c].item() - 1

        return (window, window_2, window_3, dsm_window, label)

train_mask = train_dense > 0
train_mask_t = torch.tensor(train_mask)
# Crea il dataset utilizzando l'immagine ridotta e la maschera creata in precedenza
dataset = WindowDataset(data_reduced, data_reduced_2, data_reduced_3, dsm_norm, train_mask_t, train_dense, window_size=window_size)
print(f"Numero di esempi nel dataset: {len(dataset)}")

# Esempio di accesso al primo elemento del dataset
if len(dataset) > 0:
    first_window, second_window, third_window, dsm_window, first_label, = dataset[0]
    print(f"Forma della prima finestra: {first_window.shape}")
    print(f"Label del primo esempio: {first_label}")
    print(f"Shape del dsm window: {dsm_window.shape}")



Numero di esempi nel dataset: 2817
Forma della prima finestra: torch.Size([32, 8, 8])
Label del primo esempio: 0
Shape del dsm window: torch.Size([1, 8, 8])


In [ ]:
'''import torch
from torch.utils.data import random_split

dataset_size = len(dataset)
val_size = int(0.1 * dataset_size)  # 10%
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Dimensione train: {len(train_dataset)}")
print(f"Dimensione validation: {len(val_dataset)}")
'''
train_dataset = dataset

In [ ]:
# prompt: tramite dataset vorrei creare la classe dataset e dataloader

from torch.utils.data import Dataset, DataLoader

# Crea un'istanza del dataset
hsi_train_dataset = HSI_Dataset(train_dataset, augmented=True)

print(f"Dimensione totale del dataset HSI: {len(hsi_train_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = True  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
train_dataloader = DataLoader(hsi_train_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(train_dataloader)}")


Dimensione totale del dataset HSI: 2817
Numero di batch nel DataLoader: 45


In [ ]:
'''# prompt: tramite dataset vorrei creare la classe dataset e dataloader

from torch.utils.data import Dataset, DataLoader

# Crea un'istanza del dataset
hsi_val_dataset = HSI_Dataset(val_dataset, augmented=False)

print(f"Dimensione totale del dataset HSI: {len(hsi_val_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = False  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
val_dataloader = DataLoader(hsi_val_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(val_dataloader)}")
'''

'# prompt: tramite dataset vorrei creare la classe dataset e dataloader\n\nfrom torch.utils.data import Dataset, DataLoader\n\n# Crea un\'istanza del dataset\nhsi_val_dataset = HSI_Dataset(val_dataset, augmented=False)\n\nprint(f"Dimensione totale del dataset HSI: {len(hsi_val_dataset)}")\n\n# Definisci i parametri del DataLoader\nbatch_size = batch_size\nshuffle = False  # Mischia i dati durante l\'addestramento\n\n# Crea un\'istanza del DataLoader\nval_dataloader = DataLoader(hsi_val_dataset, batch_size=batch_size, shuffle=shuffle)\n\nprint(f"Numero di batch nel DataLoader: {len(val_dataloader)}")\n'

In [ ]:
test_mask = test_dense > 0
test_mask_t = torch.tensor(test_mask)
# Crea il dataset utilizzando l'immagine ridotta e la maschera creata in precedenza
test_dataset = WindowDataset(data_reduced, data_reduced_2, data_reduced_3, dsm_norm, test_mask_t, test_dense, window_size=window_size)

In [ ]:
dataset_size = len(test_dataset)
val_size = int(0.1 * dataset_size)  # 10%
test_size = dataset_size - val_size

test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

print(f"Dimensione train: {len(train_dataset)}")
print(f"Dimensione validation: {len(val_dataset)}")

Dimensione train: 2817
Dimensione validation: 1218


In [ ]:
def collate_fn(batch):
    """
    batch: lista di tuple ((x1, x2, x3), y)
    """
    x1_batch = torch.stack([item[0][0] for item in batch]).float()
    x2_batch = torch.stack([item[0][1] for item in batch]).float()
    x3_batch = torch.stack([item[0][2] for item in batch]).float()
    dsm_batch = torch.stack([item[0][3] for item in batch]).float()
    y_batch = torch.tensor([item[1] for item in batch], dtype=torch.long)  # <-- forza long
    return (x1_batch, x2_batch, x3_batch, dsm_batch), y_batch


hsi_test_dataset = HSI_Dataset(test_dataset, augmented=False)

print(f"Dimensione totale del dataset HSI: {len(hsi_test_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = False  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
test_dataloader = DataLoader(hsi_test_dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

print(f"Numero di batch nel DataLoader: {len(test_dataloader)}")

Dimensione totale del dataset HSI: 10964
Numero di batch nel DataLoader: 172


In [ ]:
from torch.utils.data import Dataset, DataLoader

# Crea un'istanza del dataset
hsi_val_dataset = HSI_Dataset(val_dataset, augmented=False)

print(f"Dimensione totale del dataset HSI: {len(hsi_val_dataset)}")

# Definisci i parametri del DataLoader
batch_size = batch_size
shuffle = False  # Mischia i dati durante l'addestramento

# Crea un'istanza del DataLoader
val_dataloader = DataLoader(hsi_val_dataset, batch_size=batch_size, shuffle=shuffle)

print(f"Numero di batch nel DataLoader: {len(val_dataloader)}")

Dimensione totale del dataset HSI: 1218
Numero di batch nel DataLoader: 20


In [ ]:
for (x1, x2, x3, dsm_w), y in test_dataloader:
    print(y.shape, y.dtype, y.min(), y.max())
    print(x1.shape)
    print(dsm_w.shape)
    break


torch.Size([64]) torch.int64 tensor(0) tensor(14)
torch.Size([64, 32, 8, 8])
torch.Size([64, 1, 8, 8])


In [ ]:
!mkdir /content/{name}/dataset

In [ ]:
#output_path = "/content/" + name + "/dataset/labels.txt"
#with open(output_path, "w") as f:
#  f.write(f"Distribuzione labels: {labels_recap}\n")

In [ ]:
#torch.save(hsi_train_dataset.data_windows, "/content/"+ name +"/dataset/hsi_train_dataset.pt")
#torch.save(hsi_val_dataset.data_windows, "/content/"+ name +"/dataset/hsi_val_dataset.pt")
#torch.save(hsi_test_dataset.data_windows, "/content/"+ name +"/dataset/hsi_test_dataset.pt")

## Train

In [ ]:
n_classes = 16
window_size = 16

In [ ]:
vit = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    dropout_final = parameters["dropout_final"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
    window_size = parameters['window_size'],
    dsm_embedding = parameters["dsm_embedding"],
)

In [ ]:
vit(torch.rand(16, 48, 16, 16).to(device))

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
log_dir = "/content/"+name+"/logs"
csv_logger = CSVLogger(save_dir=log_dir, name="")

In [ ]:
vitgatr_mnist = ViTGATr_LIGHT(
    model = vit,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

Class weights: tensor([0.9533, 0.9937, 0.9832, 1.0043, 1.0151, 1.0376, 0.9631, 0.9884, 0.9781,
        0.9884, 1.0433, 0.9832, 1.0262, 1.0433, 1.0097])
Weight for trees (classe 0): tensor(0.9533)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.0, patience=40, verbose=False, mode="min")
'''checkpoint_callback = ModelCheckpoint(
    dirpath="/content/"+ name + "/checkpoints/",         # directory dove salvare
    filename="hci",  # nome file
    save_top_k=1,                   # salva solo il migliore (o tutti)
    monitor="loss",            # metrica da monitorare
    mode="min",                    # "min" o "max" a seconda della metrica
)'''
checkpoint_callback = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci-{epoch:02d}-{accuracy:.3f}",
    monitor="val_accuracy_overall",
    mode="max",
    save_top_k=1,   # salva tutti quelli che superano la soglia
)

checkpoint_loss = ModelCheckpoint(
    dirpath="/content/" + name + "/checkpoints/",
    filename="hci",
    monitor="val_loss",
    mode="min",
    save_top_k=1
)


In [ ]:
trainer_mnist = Trainer(logger=csv_logger, max_epochs=300, callbacks=[checkpoint_loss, checkpoint_callback])
trainer_mnist.fit(vitgatr_mnist, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                    | Type                      | Params | Mode 
-------------------------------------------------------------------------------
0  | accuracy                | MulticlassAccuracy        | 0      | train
1  | train_accuracy          | MulticlassAccuracy        | 0      | train
2  | accuracy_overall        | MulticlassAccuracy        | 0      | train
3  | accuracy_avg            | MulticlassAccuracy        | 0      | train
4  | kappa                   | MulticlassCohenKappa      | 0      | train
5  | train_accuracy_overall  | MulticlassAccuracy        | 0      | train
6  | train_accuracy_avg   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/content/geometric-algebra-transformer/gatr/utils/misc.py:235: DeprecationWarning: torch.is_autocast_cpu_enabled() is deprecated. Please use torch.is_autocast_enabled('cpu') instead. (Triggered internally at ../torch/csrc/autograd/init.cpp:609.)
  if not (torch.is_autocast_enabled() or torch.is_autocast_cpu_enabled()):


[Validation] Epoch: 0  Loss: 3.9910  Accuracy Overall: 0.0469  Accuracy Avg: 0.0578  Kappa: -0.0015


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

[Validation] Epoch: 73  Loss: 1.1427  Accuracy Overall: 0.8522  Accuracy Avg: 0.8741  Kappa: 0.8398
[Train] Epoch: 73  Loss: 0.8145  Accuracy Overall: 0.9528  Accuracy Avg: 0.9533  Kappa: 0.9494


Validation: |          | 0/? [00:00<?, ?it/s]

[Validation] Epoch: 74  Loss: 1.1455  Accuracy Overall: 0.8366  Accuracy Avg: 0.8642  Kappa: 0.8230
[Train] Epoch: 74  Loss: 0.8114  Accuracy Overall: 0.9546  Accuracy Avg: 0.9550  Kappa: 0.9513


Validation: |          | 0/? [00:00<?, ?it/s]

## Test

In [ ]:
from fvcore.nn import parameter_count
params = parameter_count(vitgatr_mnist)

print(f"Params: {params[''] / 1e6:.2f} M")

In [ ]:
trainer_mnist.test(vitgatr_mnist, dataloaders=test_dataloader, verbose=False)
results_last = vitgatr_mnist.test_results
print(results_last)

accuracies_last = vitgatr_mnist.test_per_class_accuracies

In [ ]:
vit_l = ViGatr(
    in_channels = parameters["in_channels"],
    out_channels = parameters["out_channels"],
    blade = blade,
    blade_len = blade.shape[0],
    hidden_dim = parameters["hidden_dim"],
    n_heads = parameters["n_heads"],
    n_classes = n_classes,
    crop_size = window_size // 4,
    positional_dim = parameters["positional_dim"],
    mv_in_channels = parameters["mv_in_channels"],
    blocks = parameters["blocks"],
    dropout_gatr = parameters["dropout_gatr"],
    dropout_final = parameters["dropout_final"],
    stride = parameters["stride"],
    type_embedding = parameters["type_embedding"],
    window_size = parameters['window_size'],
    dsm_embedding = parameters["dsm_embedding"],
)

In [ ]:
vitgatr_mnist_l = ViTGATr_LIGHT.load_from_checkpoint(
    "/content/" + name + "/checkpoints/hci.ckpt",
    #"/content/test-salinas-22-07-N0/checkpoints/hci-epoch=54-accuracy=0.000.ckpt",
    model = vit_l,
    n_classes = n_classes,
    multivector_labels = multivector_labes,
    learning_rate = parameters["lr"],
)

In [ ]:
trainer_mnist.test(vitgatr_mnist_l, dataloaders=test_dataloader, verbose=False)
results_loaded = vitgatr_mnist_l.test_results
print(results_loaded)

accuracies_loaded = vitgatr_mnist_l.test_per_class_accuracies

In [ ]:
output_path = "/content/" + name + "/results.txt"
with open(output_path, "w") as f:
    f.write(f"Params: {params[''] / 1e6:.3f} M\n\n")
    f.write(f"Results last\n")
    f.write(f"Accuracy Overall: {results_last[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_last[1]:.4f}\n")
    f.write(f"Kappa: {results_last[2]:.4f}\n\n")
    f.write(f"Results loaded\n")
    f.write(f"Accuracy Overall: {results_loaded[0]:.4f}\n")
    f.write(f"Accuracy Avg: {results_loaded[1]:.4f}\n")
    f.write(f"Kappa: {results_loaded[2]:.4f}\n")

    f.write(f"\nLast\n")
    for i, acc in enumerate(accuracies_last):
        f.write(f"Classe {i} ({vitgatr_mnist.class_names[i]}): {acc:.4f}\n")

    f.write(f"\nLoad\n")
    for i, acc in enumerate(accuracies_loaded):
        f.write(f"Classe {i} ({vitgatr_mnist_l.class_names[i]}): {acc:.4f}\n")


## Download

In [ ]:
!cp -r /content/{name} /content/drive/MyDrive/tesi